In [29]:
%%javascript
 IPython.OutputArea.prototype._should_scroll = function(lines) {
     return false;
 }

<IPython.core.display.Javascript object>

In [30]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [31]:
# Imports
# pip installs: geojson, pandas, ipyleaflet, svgpath2mpl, bokeh, openpyxl

import geojson, re, pickle
import pandas as pd
import numpy as np
from ipywidgets import widgets, Layout, ToggleButtonsStyle
from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoJSON, Polygon, Popup
from datetime import datetime
from matplotlib.patches import RegularPolygon
import xml.etree.ElementTree as etree
from svgpath2mpl import parse_path
from bokeh.io import output_notebook
from bokeh.io import show, curdoc
from bokeh.plotting import figure
from bokeh import events
from bokeh.models import CustomJS, Div, TapTool, ColumnDataSource, MultiPolygons, Plot, LinearAxis, Grid
from bokeh.layouts import column as bkCol
from bokeh.layouts import row as bkRow

In [32]:
output_notebook()

Loading BokehJS ...

In [33]:
# UK Constituency classes

class CONSTITUENCY():
    
    const_list = {}
    previous_list = {}
    
    def __init__(self,data):
        self.name = data['Name'] # string
        self.region = data['Region'] # string
        self.county = data['County'] # string
        self.created = get_election_date_from_string(data['Created']) # datetime
        self.predecessors1 = data['Original Predecessors'] # string
        self.abolished = self.split_dates(data['Abolished']) # list
        self.successors = data['Successors'] # string
        self.recreated = self.split_dates(data['Re-created']) # list
        self.predecessors2 = data['Predecessors'] # string
        self.previous = data['Previous Name'] # string
        self.changed = self.split_dates(data['Changed']) # datetime
        self.quadMP = self.split_dates(data['4 MPs']) # datetime
        self.tripleMP = self.split_dates(data['3 MPs']) # datetime
        self.doubleMP = self.split_dates(data['2 MPs']) # datetime
        self.singleMP = self.split_dates(data['1 MP']) # datetime
        self.election_list = {} # dictionary
        CONSTITUENCY.const_list.update({self.name:self})
        if self.previous != '':
            for name in self.previous.split('|'):
                CONSTITUENCY.previous_list.update({name:self})
    
    def split_dates(self,date_string):
        
        if date_string == '':
            return []
        
        if isinstance(date_string,int):
            date_string = str(date_string)
        
        date_list = date_string.split('|')
        
        date_list = [get_election_date_from_string(x) for x in date_list]
        
        return date_list
    
    def update_results(self,date_string,res_list):
        
        date = get_election_date_from_string(date_string)
        
        if not self.existed(date):
            print("Results for when const didn't exist",self.name,date_string)
            raise Exception
        
        seats = self.get_seats(date)
        
        if date_string[-1] == 'B':
            if date_string[-2] != 'B':
                if seats == 2 and not self._has_ex_mp(res_list):
                    print("No candidate for by-election with two seats",self.name,date_string)
                    raise Exception
                seats = 1
            res = CONST_RESULT(res_list,seats,True)
            self.election_list.update({date_string:res})            
        else:
            res = CONST_RESULT(res_list,seats,False)
            self.election_list.update({date_string:res})
            
    def get_seats(self,date):
        
        if self.singleMP != [] and date >= self.singleMP[0] and date < self.singleMP[1]:
            return 1
        elif self.doubleMP != [] and date >= self.doubleMP[0] and date < self.doubleMP[1]:
            return 2
        elif self.tripleMP != [] and date >= self.tripleMP[0] and date < self.tripleMP[1]:
            return 3
        elif self.quadMP != [] and date >= self.quadMP[0] and date < self.quadMP[1]:
            return 4
        else:
            return 1
    
    def _has_ex_mp(self,res_list):
        
        if len(res_list) == 1:
            
            if res_list[0]['Notes'][0] != '[':
                return False
            else:
                return True
        
        for res in res_list:
            if res['Party'] == 'Turnout':
                if res['Notes'][0] != '[':
                    return False
                else:
                    return True
    
    def existed(self,date):
        
        if date < self.created:
            return False
        
        if len(self.abolished) > 0 and len(self.recreated) == 0:
            
            if date > self.abolished[0]:
                return False
            
        elif len(self.abolished) > 0 and len(self.recreated) > 0:
            
            for i in range(0,len(self.abolished)):
                if i+1 > len(self.recreated) and date >= self.abolished[i]:
                    return False
                if date >= self.abolished[i] and date < self.recreated[i]:
                    return False
        
        return True
        
    def get_mps(self,date):
        
        elections = list(self.election_list.keys())
        elections = {get_election_date_from_string(x):x for x in elections}
        last_election = ''
        for election in sorted(elections.keys()):
            if date < election:
                if last_election == '':
                    if self.existed(date):
                        mp_list = "Constituency existed but results not available"
                    else:
                        mp_list = "Constituency not in existance"
                    break
                if self.get_seats(get_election_date_from_string(elections[last_election])) == 1:
                    mp_list = self.election_list[elections[last_election]].winner1
                    break
                else:
                    w1 = self.election_list[elections[last_election]].winner1
                    w2 = self.election_list[elections[last_election]].winner2
                    if w2 == '':
                        lw1 = self.election_list[elections[last_last_election]].winner1
                        lw2 = self.election_list[elections[last_last_election]].winner2
                        notes = self.election_list[elections[last_election]].notes
                        ex_mp = notes.split(']')[0].replace('[','')
                        if lw1.split(' ')[-1] == ex_mp:
                            w2 = lw2
                        else:
                            w2 = lw1
                    mp_list = w1 + ", " + w2
                    break
            if last_election == '':
                last_last_election = ''
                last_election = election
            else:
                last_last_election = last_election
                last_election = election
        
        return mp_list
    
    def check_elections(self):
        
        pass
    
    def get_winners(consts,election,election_type='',mode='party'):
        
        winners = []
        
        for const in consts:

            if const not in CONSTITUENCY.const_list.keys():
                const_obj = CONSTITUENCY.previous_list[const]
            else:
                const_obj = CONSTITUENCY.const_list[const]

            if mode == 'party':

                try:
                    results = const_obj.election_list[election]
                except:
                    raise Exception('Election results not found',const,election)

                winners += results.parties
                
        return winners

class CONST_RESULT():
    
    def __init__(self,results,seats,byelection):
        self.results = results
        self.winners = []
        self.parties = []
        self.winnerN = []
        self.winnerP = []
        self.majority = 0
        self.unopposed = False
        self.turnoutN = 0
        self.turnoutP = 0
        self.notes = ''
        self.seats = seats
        self.overturned = False
        self.byelection = byelection
        self.process_results()
        
    def process_results(self):
        
        df_list = []
        
        if len(self.results) == self.seats:
            for i in range(0,self.seats):
                self.winners.append(self.results[i]['Candidate'])
                self.parties.append(self.results[i]['Party'])
                if self.results[i]['Notes'] != '':
                    self.notes = self.results[i]['Notes']
            self.unopposed = True
            self.results = pd.DataFrame(self.results)            
        else:
            ot_cand = []
            for res in self.results:
                if res['Party'] == 'Turnout':
                    self.turnoutN = res['Votes']
                    self.turnoutP = res['Percent']
                    self.notes = res['Notes']            
                else:
                    if isinstance(res['Votes'],str) and '*' in res['Votes']:
                        res['Votes'] = int(res['Votes'].replace('*',''))
                        self.overturned = True
                        ot_cand.append(res['Candidate'])
                    df_list.append(res)
                    
            self.results = pd.DataFrame(df_list)
            self.results.sort_values('Votes',ascending=False)
            self.results.reset_index(inplace=True)
            
            i = 0
            j = 0
            while i < self.seats:
                if self.results['Candidate'][j] not in ot_cand:
                    self.winners.append(self.results['Candidate'][j])
                    self.parties.append(self.results['Party'][j])                
                    self.winnerN.append(self.results['Votes'][j])
                    self.winnerP.append(self.results['Percent'][j])
                    if i == 0 and self.seats == 1 and not self.unopposed:
                        self.majority = self.results['Votes'][j]
                    elif i == 1 and self.seats == 1 and not self.unopposed:
                        self.majority -= self.results['Votes'][j]
                    i += 1
                j += 1

def get_election_date_from_string(date_string):
    
    try:
        if date_string == '':
            return ''

        if not isinstance(date_string,str):
            if date_string < 1830 or date_string == 1861: # Birkenhead created between elections
                return datetime.strptime(str(date_string),'%Y')
            date_string = str(int(date_string))

        byelection = False

        if date_string[-1] == 'B':
            date_string = date_string[:-1]
            byelection = True
            if date_string[-1] == 'B':
                date_string = date_string[:-1]

        if byelection:

            if len(date_string) == 4:
                return datetime.strptime(date_string,'%Y')
            elif len(date_string) == 8:
                return datetime.strptime(date_string,'%Y %b')
            else:
                return datetime.strptime(date_string,'%Y %d%b')
        else:

            return ELECTION.elect_list[date_string].start_date
    
    except:
        print(date_string)
        raise Exception


In [34]:
# Other UK classes

class ELECTION():
    
    elect_list = {}
    year_list = set()
    map_list = []
    hex_list = []
    centre_point = (54, 0)
    hex_df = pd.read_excel("UK_Elections.xlsm",sheet_name="Hex")
    hex_df.fillna('',axis=1,inplace=True)
    division_name = 'Constituency'
    hex_aspect = 1
    svg_aspect = 0.7
    svg_data = {}
    
    def __init__(self,data):
        self.year = str(data['Year'])
        self.start_date, self.end_date = self.parse_dates(data['Date'])
        self.largest_party = data['Largest Party']
        self.pm = data['Prime Minister']
        self.turnout = data['Turnout']
        self.const_list = []
        self.result = {}
        self.result_df = ''
        self.party_consts = {}
        self.total_votes = 0
        self.map = data['Map File']
        if self.map:
            ELECTION.map_list.append(self.year)
            ELECTION.map_list = list(set(ELECTION.map_list))
        self.hex = data['Hex Col']
        if self.hex:
            ELECTION.hex_list.append(self.year)
            ELECTION.hex_list = list(set(ELECTION.hex_list))
        ELECTION.elect_list.update({self.year:self})
        ELECTION.year_list.add(self.year[:4])
        
    def parse_dates(self,date_string):
        
        if not isinstance(date_string,datetime):
            date_string = date_string.split('-')
            return datetime.strptime(date_string[0],'%d/%m/%Y'), datetime.strptime(date_string[1],'%d/%m/%Y')
        else:
            return date_string, ''

    def get_consts(self):
        
        self.const_list = []
        
        for const in CONSTITUENCY.const_list.values():
            if self.year in const.election_list.keys():
                self.const_list.append(const)
        
    def get_results(self):
                
        if self.const_list == []:
            self.get_consts()
        
        for const in self.const_list:
            
            res = const.election_list[self.year]
            
            for row in res.results.index:
                party = self.check_alliance(res.results['Party'][row])
                if party not in self.result.keys():
                    self.result[party] = {'Votes':0,'Seats':0,'Percent':0}
                if res.results['Votes'][row] != 'Unopposed':
                    try:
                        self.result[party]['Votes'] += res.results['Votes'][row]
                        self.total_votes += res.results['Votes'][row]
                    except:
                        raise Exception(const.name,self.year)
            
            for i in range(0,len(res.parties)):
                party = self.check_alliance(res.parties[i])
                if party not in self.result.keys():
                    self.result[party] = {'Votes':0,'Seats':0,'Percent':0}
                self.result[party]['Seats'] += 1
                if party not in self.party_consts.keys():
                    self.party_consts[party] = []
                self.party_consts[party].append(const.name)
        
        for party in self.result.keys():
            self.result[party]['Percent'] = self.result[party]['Votes'] / self.total_votes * 100
            
        sum_dict = {'Party':[],'Seats':[],'Votes':[],'Percent':[]}
        
        for party in self.result.keys():
            sum_dict['Party'].append(party)
            sum_dict['Seats'].append(self.result[party]['Seats'])
            sum_dict['Votes'].append(self.result[party]['Votes'])
            sum_dict['Percent'].append(self.result[party]['Percent'])
        
        df = pd.DataFrame(sum_dict)
        df.sort_values(by='Party',inplace=True)
        df.sort_values(by=['Seats','Votes'],inplace=True,ascending=False)
        self.result_df = df
    
    def check_alliance(self,party):
        
        if party in PARTY.party_list.keys() and PARTY.party_list[party].coalition != '':
            return PARTY.party_list[party].coalition
        
        return party

class PARTY():
    
    party_list = {}
    
    def __init__(self,name,colour,colour_scale,coalition=False):
        self.name = name
        self.colour = colour
        self.colour_scale = colour_scale.split(',')
        self.coalition = coalition
        PARTY.party_list[self.name] = self

def check_results(df):
    
    for row in df.index:
        const = df['Constituency'][row]
        year = df['Year'][row]
        if year[-1] == 'B':
            if year[:4] in ELECTION.year_list and len(year) < 9:
                print("By-election in general election year but no month given")
                print(row,year,const)
                raise Exception
        else:
            if year not in ELECTION.elect_list.keys():
                print("General election not in election list")
                print(row,year,const)
                raise Exception

def process_consts(df):
    
    for row in df.index:
        CONSTITUENCY(df.iloc[row].to_dict())
        
def process_elections(df):
    
    for row in df.index:
        ELECTION(df.iloc[row].to_dict())
        
def process_results(df):
    
    log = widgets.HTML("")
    display(log)
    
    df['Notes'].fillna('',inplace=True)
    
    const_elect = {}
    
    for row in df.index:
        const = df['Constituency'][row]
        year = str(df['Year'][row])
        if const not in const_elect.keys():
            const_elect[const] = {}
        if year not in const_elect[const].keys():
            const_elect[const][year] = []
        if df['Votes'][row] != 'Unopposed':
            if not isinstance(df['Votes'][row],int):
                try:
                    int(df['Votes'][row].replace(",","").replace("*",""))
                except:
                    print("Vote not integer",const,year)
                    raise Exception
            if not isinstance(df['Percent'][row],float) and not isinstance(df['Percent'][row],int):
                try:
                    float(df['Percent'][row].replace(" (est)",""))
                except:
                    print("Percent not float",const,year)
                    raise Exception
        if df['Party'][row] in PARTY.party_list.keys():
            colour = PARTY.party_list[df['Party'][row]].colour
        else:
            colour = '#C8C7C9'
        elect_dict = {
                      'Party': df['Party'][row],
                      'Party Colour': colour,
                      'Candidate': df['Candidate'][row],
                      'Votes': df['Votes'][row],
                      'Percent': df['Percent'][row],
                      'Notes': df['Notes'][row]
                     }
        const_elect[const][year].append(elect_dict)
    
    for const in const_elect.keys():
        const_obj = CONSTITUENCY.const_list[const]
        for year in const_elect[const].keys():
            if year[-1] == 'B':
                byelection = True
                if year[-2] == 'B':
                    byyear = year[:-2]
                else:
                    byyear = year[:-1]
            else:
                byelection = False
            if byelection and byyear in ELECTION.year_list and len(byyear) == 4:
                print("Byelection in gen year without month",const,year)
                raise Exception
            if not byelection and year not in ELECTION.elect_list.keys():
                print("Not election year",const,year)
                raise Exception
            turnout = False
            notes = False
            for res in const_elect[const][year]:
                if res['Party'] == 'Turnout' and turnout:
                    print("Multiple Turnout",const,year)
                    raise Exception
                elif res['Party'] == 'Turnout':
                    turnout = True
                if res['Notes'] != '':
                    notes = True
            vote_set = list(set([x['Votes'] for x in const_elect[const][year]]))
            if len(vote_set) == 1 and vote_set[0] == 'Unopposed':
                turnout = True
            
            if not turnout:
                print("Missing turnout",const,year)
                raise Exception
            if byelection and not notes:
                print("Byelection no notes",const,year)
                raise Exception
            
            log.value = str(const) + " " + str(year)
            const_obj.update_results(year,const_elect[const][year])

def process_parties(df):
    
    df.fillna('',axis=1,inplace=True)
    for row in df.index:
        PARTY(df.Party[row],df.Colour[row],df['Colour Scale'][row],df['Main Party'][row])


In [35]:
# Convert svg into coordinates
def convert_svg(filename,country,election,us_election=''):

    def parse_svg(root,election):

        def transform_path(path,transform,election=''):
            
            if transform[:6] == 'matrix':
                a,b,c,d,e,f = [float(i) for i in transform[7:-1].split(',')]
                count = 0
                for coords in path.vertices:
                    x = a * coords[0] + c * coords[1] + e
                    y = b * coords[0] + d * coords[1] + f
                    path.vertices[count][0] = x
                    path.vertices[count][1] = y
                    count += 1

            elif transform[:9] == 'translate':
                coords_split = [float(i) for i in transform[10:-1].split(',')]
                if len(coords_split) == 2:
                    x_shift, y_shift = coords_split
                count = 0
                for coords in path.vertices:
                    x = coords[0] + x_shift
                    y = coords[1] + y_shift
                    path.vertices[count][0] = x
                    path.vertices[count][1] = y
                    count += 1

            elif transform == 'Shetland':

                if election == '1997':
                    a,b,c,d,e,f = [0.7,0,0,0.7,-60,-180]
                elif election == '1992':
                    a,b,c,d,e,f = [0.3,0,0,0.3,60,-190]
                elif election == '1979':
                    a,b,c,d,e,f = [0.7,0,0,0.7,0,-280]
                elif election == '1970':
                    a,b,c,d,e,f = [1.5,0,0,1.5,-200,-1000]

                count = 0
                for coords in path.vertices:
                    x = a * coords[0] + c * coords[1] + e
                    y = b * coords[0] + d * coords[1] + f
                    path.vertices[count][0] = x
                    path.vertices[count][1] = y
                    count += 1 

            return path

        def check_element(paths, names, element, transform, election):

            if 'transform' in element.keys():
                transform = element.attrib['transform']

            if element.tag != '{http://www.w3.org/2000/svg}path':
                for e in element:
                    paths, names = check_element(paths,names,e,transform,election)
            else:
                if 'd' in element.keys() and element.attrib['d'] != '':

                    path = element.attrib['d']
                    if path[-1] not in ('z','Z') and path[-2] not in ('z','Z'):
                        path = path + 'z'
                    try:
                        parsed = parse_path(path)
                    except:
                        print(element.attrib['id'])
                        parsed = parse_path(path)
                    names.append(element.attrib['id'])
                    if element.attrib['id'] == 'Orkney and Shetland':
                        parsed = transform_path(parsed,'Shetland',election=election)
                    elif transform:
                        parsed = transform_path(parsed,transform)
                    if parsed:
                        paths.append(parsed)

                    return paths, names

            return paths, names

        tree = etree.parse(filename)
        root = tree.getroot()
        paths = []
        names = []

        for element in root:
            transform = None
            paths, names = check_element(paths, names, element,transform,election)

        return paths, names

    def convert_paths(paths, names, election):

        def convert_path_into_polygons(path,pg_x,pg_y,count=-1):

            if count == -1:

                vertices = path.vertices
                codes = path.codes

                cur_pg = []

                for i in range(0,len(vertices)):
                    if codes[i] == 79:
                        pg_x.append([[c[0] for c in cur_pg]])
                        pg_y.append([[c[1]*-1 for c in cur_pg]]) # multiply by -1 to invert
                        cur_pg = []
                    else:
                        cur_pg.append(vertices[i])

            elif names[count] == 'Milton Keynes North East':

                vertices = []
                vertices += list(path.vertices[:6])
                vertices += list(paths[count+1].vertices[1:10])
                vertices += list(path.vertices[15:])
                codes = [1,]
                codes += [2] * (len(vertices)-2) 
                codes.append(79)

                cur_pg = []
                names[count] = 'Milton Keynes'

                for i in range(0,len(vertices)):
                    if codes[i] == 79:
                        pg_x.append([[c[0] for c in cur_pg]])
                        pg_y.append([[c[1]*-1 for c in cur_pg]]) # multiply by -1 to invert
                        cur_pg = []
                    else:
                        cur_pg.append(vertices[i])

            return pg_x, pg_y

        xs = []
        ys = []
        x_temp = []
        y_temp = []
        anomalies = {}
        count = 0

        for path in paths:

            if 'Milton' in names[count] and election in ('1987','1983'):
                x_temp, y_temp = convert_path_into_polygons(path,x_temp,y_temp,count)
                if names[count] == 'Milton Keynes':
                    xs.append(x_temp)
                    ys.append(y_temp)
                x_temp = []
                y_temp = []
            elif election in ('1992','1987','1983') and names[count] in ('path4893','Glasgow Central'):
                anomalies[names[count]] = path
            else:    
                x_temp, y_temp = convert_path_into_polygons(path,x_temp,y_temp)
                if count == len(paths)-1 or names[count+1] != names[count]:       
                    xs.append(x_temp)
                    ys.append(y_temp)
                    x_temp = []
                    y_temp = []

            count += 1

        return xs, ys, anomalies

    def correct_glasgow(names,xs,ys,anomalies):

        names.remove('Glasgow Central')
        names.remove('path4893')
        names.append('Glasgow Central')

        vertices = []
        vertices += list(anomalies['path4893'].vertices[0:11])
        vertices += list(anomalies['Glasgow Central'].vertices[0:6])
        vertices += list(anomalies['path4893'].vertices[16:])
        codes = [1,]
        codes += [2] * (len(vertices)-2) 
        codes.append(79)

        cur_pg = []

        for i in range(0,len(vertices)):
            if codes[i] == 79:
                xs.append([[[c[0] for c in cur_pg]]])
                ys.append([[[c[1]*-1 for c in cur_pg]]]) # multiply by -1 to invert
                cur_pg = []
            else:
                cur_pg.append(vertices[i])

        return names, xs, ys

    def check_for_holes(xs, ys, election):

        const_holes = {
            '1970':
                   {'PENRITH AND THE BORDER':['CARLISLE'],
                    'DON VALLEY':['DONCASTER'],
                    'CIRENCESTER AND TEWKESBURY':['CHELTENHAM'],
                    'NORTH SOMERSET':['BATH'],
                    'CAMBRIDGESHIRE':['CAMBRIDGE'],
                    'CENTRAL NORFOLK':['NORWICH NORTH','NORWICH SOUTH']},
            '1979':
                   {'PENRITH AND THE BORDER':['CARLISLE'],
                    'CIRENCESTER AND TEWKESBURY':['CHELTENHAM'],
                    'NORTH SOMERSET':['BATH'],
                    'CAMBRIDGESHIRE':['CAMBRIDGE'],
                    'DAVENTRY':['NORTHAMPTON NORTH','NORTHAMPTON SOUTH']},
            '1992':
                   {'PENRITH AND THE BORDER':['CARLISLE'],
                    'WANSDYKE':['BATH'],
                    'CIRENCESTER AND TEWKESBURY':['CHELTENHAM']},
            '1997':
                   {'NORTH ESSEX':['COLCHESTER']}
                  }

        for key in const_holes.keys():

            if election == key:

                for const_key in const_holes[key].keys():
                    
                    for const in const_holes[key][const_key]:
                        count = 0
                        outer = 0
                        inner = 0
                        
                        for name in names:
                            if name.upper() == const_key:
                                outer = count
                            elif name.upper() == const:
                                inner = count
                            count += 1
                        
                        xs[outer][0].append(xs[inner][0][0])
                        ys[outer][0].append(ys[inner][0][0])

        return xs, ys

    tree = etree.parse(filename)
    root = tree.getroot()
    path_elems = root.findall('.//{http://www.w3.org/2000/svg}path')
    paths, names = parse_svg(root, election)
    
    xs, ys, anomalies = convert_paths(paths, names, election)
    
    
    if election in ('1987','1983'):
        names.remove('Milton Keynes South West')
    if election in ('1992','1987','1983'):
        names, xs, ys = correct_glasgow(names,xs,ys,anomalies)

    for i in range(len(names)-1,-1,-1):
        if i != 0 and names[i]==names[i-1]:
            names.pop(i)
    
    xs, ys = check_for_holes(xs,ys,election)
    
    return xs, ys, names


In [24]:
# # Load UK data from Excel

# election_df = pd.read_excel("UK_Elections.xlsm",sheet_name="Elections")
# election_df.fillna('',axis=1,inplace=True)
# process_elections(election_df)

# party_df = pd.read_excel("UK_Elections.xlsm",sheet_name="Parties")
# process_parties(party_df)

# const_sum_df = pd.read_excel("UK_Elections.xlsm",sheet_name="Const_sum")
# const_sum_df = const_sum_df.fillna('')

# for row in const_sum_df.index:
#     CONSTITUENCY(const_sum_df.loc[row])
    
# const_df = pd.read_excel("UK_Elections.xlsm",sheet_name="Const_full")
# process_results(const_df)

# for year in ELECTION.elect_list.keys():
#     ELECTION.elect_list[year].get_results()

# # Process SVGs
# uk_input_dict = [{'filename':'SVGs/UK1997.svg','elections':['1997','2001']},
#                  {'filename':'SVGs/UK1992.svg','elections':['1992','1987','1983']},
#                  {'filename':'SVGs/UK1979.svg','elections':['1979','1974 Oct','1974 Feb']},
#                  {'filename':'SVGs/UK1970.svg','elections':['1970','1966','1964','1959','1955']}]

# for group in uk_input_dict:
#     xs, ys, names = convert_svg(group['filename'],'UK',group['elections'][0])
#     for election in group['elections']:
        
#         if 'test' in group.keys():
#             ELECTION.svg_data[election] = {'xs':xs,'ys':ys,'names':names,'test':True}
#         else:
#             ELECTION.svg_data[election] = {'xs':xs,'ys':ys,'names':names,'test':False}    

# # Save data
# with open("uk_pickled_data", "wb") as f:
#     election_dict = {'elect':ELECTION.elect_list,
#                      'year':ELECTION.year_list,
#                      'map':ELECTION.map_list,
#                      'hex':ELECTION.hex_list,
#                      'svg':ELECTION.svg_data}
#     const_dict = {'main':CONSTITUENCY.const_list,
#                   'prev':CONSTITUENCY.previous_list}
#     saved_dict = {"consts":const_dict,"elections":election_dict,"parties":PARTY.party_list}
    
#     pickle.dump(saved_dict,f)

HTML(value='')

In [36]:
# Load UK data from pickle

with open("uk_pickled_data", "rb") as f:
    saved_dict = pickle.load(f)

CONSTITUENCY.const_list, CONSTITUENCY.previous_list = saved_dict["consts"].values()
ELECTION.elect_list, ELECTION.year_list, ELECTION.map_list, ELECTION.hex_list, ELECTION.svg_data = saved_dict["elections"].values()
PARTY.party_list = saved_dict["parties"]

# with open('Data/Can/canED.geojson') as f:
#     can_gj = geojson.load(f)

# uk_hex_df = pd.read_excel("UK_Elections.xlsm",sheet_name="Hex")
# uk_hex_df.fillna('',axis=1,inplace=True)

In [37]:
# US Classes

class STATE():
    
    state_list = {}
    
    def __init__(self,data):
        
        self.name = data['State']
        self.abbr = data['Abbr']
        self.pres_results = {}
        if self.name != 'District of Columbia':
            self.order = data['Statehood Order']
            self.year = str(data['Statehood Year'])
            self.senate_classes = (data['Class 1 Senator'],data['Class 2 Senator'],data['Class 3 Senator'])
            self.house_seats = {}
            self.district_list = {}
            self.senate_results = {}
            other_cols = ['State','Abbr','Statehood Order','Statehood Year','Class 1 Senator','Class 2 Senator','Class 3 Senator']
            census_list = [col for col in data.index if col not in other_cols]
            for census in census_list:
                self.house_seats[census] = data[census]
        STATE.state_list[self.name] = self
        
    def get_allocation(self,year,inst):
        
        year = int(year)
        seats = 0
        
        if year >= self.year:
            for census in self.house_seats.keys():
                if year >= census:
                    seats = self.house_seats[census]
        else:
            return 0
        
        if inst == 'house':
            return seats
        else:
            return seats + 2
    
    def process_senate_results(self,results,year,special=False):
        
        results.sort_values(by='Votes',inplace=True,ascending=False)
        results.reset_index(inplace=True,drop=True)
        senate_class = US_ELECTION.elect_list[year].senate_class
        res = SENATE_RESULT(results,senate_class,special)
        if year in self.senate_results.keys():
            self.senate_results[year].append(res)
        else:
            self.senate_results[year] = [res]
    
    def process_pres_results(self,results,year):
        
        results.sort_values(by='Votes',inplace=True,ascending=False)
        results.reset_index(inplace=True,drop=True)
        res = PRES_RESULT(results)
        self.pres_results[year] = res

    def get_winners(states,election,election_type,mode='party'):
        
        winners = []
        
        for state in states:
            
            state_obj = STATE.state_list[state]

            if mode == 'party':

                try:
                    if election_type == 'Senate':
                        norm_found = False
                        results = state_obj.senate_results[election]
                        for res in results:
                            if not res.special:
                                winners.append(res.party)
                                norm_found = True
                        if not norm_found:
                            winners.append('No Election')
                    else:
                        results = state_obj.pres_results[election]
                        winners.append(results.party)
                except:
                    winners.append('No Election')
        
        return winners

class DISTRICT():
    
    const_list = {}
    
    def __init__(self,data):
        
        self.state = data['State']
        self.name = data['District']
        self.created = int(data['Created'])
        if data['Abolished'] == '':
            self.abolished = []
        else:
            self.abolished = [int(x) for x in str(data['Abolished']).split('|')]
        if data['Re-created'] == '':
            self.recreated = []
        else:
            self.recreated = [int(x) for x in str(data['Re-created']).split('|')]
        self.multi_seats = data['Multiple Seats'].split(',')
        self.election_list = {}
        STATE.state_list[self.state].district_list[self.name] = self
        DISTRICT.const_list[self.name] = self
        
    def existed(self,year):
        
        if year < self.created:
            return False
        
        if len(self.abolished) > 0 and len(self.recreated) == 0:
            
            if year > self.abolished[0]:
                return False
            
        elif len(self.abolished) > 0 and len(self.recreated) > 0:
            
            for i in range(0,len(self.abolished)):
                if i+1 > len(self.recreated) and year >= self.abolished[i]:
                    return False
                if year >= self.abolished[i] and year < self.recreated[i]:
                    return False
        
        return True

    def get_seats(self,year):
        
        year = int(year)
        
        if self.multi_seats != ['']:
            for block in self.multi_seats:
                seats = int(block.split(':')[1])
                start_date = int(block.split(':')[0].split('|')[0])
                end_date = int(block.split(':')[0].split('|')[1])
                if year >= start_date and year < end_date:
                    return seats
            
        return 1
    
    def add_results(self,data):
        
        for year in list(data['Year'].unique()):
            year_df = data[data['Year']==year]
            year_df.sort_values(by='Votes',inplace=True,ascending=False)
            year_df.reset_index(inplace=True,drop=True)
            year_df['Year'] = year_df['Year'].apply(str)
            seats = self.get_seats(year)
            res = DISTRICT_RESULT(seats,year_df)
            self.election_list.update({str(year):res})

    def get_winners(districts,election,election_type='',mode='party'):
        
        winners = []
        
        for district in districts:
            
            dist_obj = DISTRICT.const_list[district]

            if mode == 'party':

                try:
                    results = dist_obj.election_list[election]
                except:
                    raise Exception('Election results not found',district,election)

                winners.append(results.party1)
                
        return winners

class DISTRICT_RESULT():
    
    def __init__(self,seats,results,byelection=False):
        self.results = results
        self.winner1 = ''
        self.party1 = ''
        self.winners = []
        self.parties = {}
        self.winnerN = 0
        self.winnersN = []
        self.winnerP = 0
        self.winnersP = []
        self.majority = 0
        self.unopposed = False
        self.turnoutN = 0
        self.turnoutP = 0
        self.notes = ''
        self.seats = seats
        self.overturned = False
        self.byelection = byelection
        self.process_results(results)
    
    def process_results(self,results):
        
        if self.seats == 1:
            self.party1 = results.loc[0,'Party']
            self.winner1 = results.loc[0,'Candidate']
            self.winnerN = results.loc[0,'Votes']
            self.winnerP = results.loc[0,'Percent']
            if self.winnerN == 'Unopposed':
                self.unopposed = True
            else:
                self.turnoutN = sum(list(results['Votes']))
        else:
            parties = list(year_df['Party'][:seats])
            self.parties = {party:parties.count(party) for party in parties}
            self.winners = list(results['Candidate'][:seats])
            self.winnersN = list(results['Votes'][:seats])

class SENATE_RESULT():
    
    def __init__(self,results,senate_class,special):
        self.results = results
        self.winner = ''
        self.party = ''
        self.winnerN = 0
        self.winnersN = []
        self.unopposed = False
        self.turnoutN = 0
        self.turnoutP = 0
        self.notes = ''
        self.overturned = False
        self.special = special
        self.senate_class = senate_class
        self.process_results(results)
    
    def process_results(self,results):
        
        self.party = results.loc[0,'Party']
        self.winner = results.loc[0,'Candidate']
        self.winnerN = results.loc[0,'Votes']
        self.winnerP = results.loc[0,'Percent']
        if self.winnerN == 'Unopposed':
            self.unopposed = True
        else:
            self.turnoutN = sum(list(results['Votes']))

class PRES_RESULT():
    
    def __init__(self, results):
        self.results = results
        self.party = self.results['Party'][0]
            
class US_ELECTION():
    
    elect_list = {}
    president_list = {}
    map_list = []
    hex_list = []
    centre_point = (30, -100)
    house_hex_df = pd.read_excel("US_Elections.xlsx",sheet_name="Dist_Hex")
    house_hex_df.fillna('',axis=1,inplace=True) 
    house_hex_aspect = 1.5
    state_hex_df = pd.read_excel("US_Elections.xlsx",sheet_name="State_Hex")
    state_hex_df.fillna('',axis=1,inplace=True)  
    state_hex_aspect = 2
    svg_aspect = 1.5
    division_name = {'House':'District','Senate':'State','President':'State'}
    svg_data = {}
    
    def __init__(self,data):
        
        self.year = str(data['Election Year'])
        self.congress = data['Congress']
        self.congress_start = data['Congress began']
        self.congress_end = data['Congress ended']
        self.house_seats = data['House Seats']
        self.map = data['Geo File']
        if self.map:
            US_ELECTION.map_list.append(self.year)
        self.pres_hex = data['Pres Hex Col']
        if self.pres_hex:
            US_ELECTION.hex_list.append(self.year)
            US_ELECTION.hex_list = list(set(US_ELECTION.hex_list))        
        self.state_hex = data['State Hex Col']
        if self.state_hex:
            US_ELECTION.hex_list.append(self.year)
            US_ELECTION.hex_list = list(set(US_ELECTION.hex_list))
        self.house_hex = data['House Hex Col']
        if self.house_hex:
            US_ELECTION.hex_list.append(self.year)
            US_ELECTION.hex_list = list(set(US_ELECTION.hex_list))
        self.senate_seats = data['Senate Seats']
        self.senate_class = data['Senate Class']
        self.house_results = {}
        self.senate_results = {}
        self.president_results = {}
        US_ELECTION.elect_list[self.year] = self
        if data['Presidential Dates'] == '':
            self.presidential = False
        else:
            self.presidential =True
            self.pres_dates = data['Presidential Dates']
            self.winning_party = data['Winning Party']
            self.democratic = data['Democratic Candidate']
            self.republican = data['Republican Candidate']
            self.dem_rep = data['Democratic-Republican Candidate(s)']
            self.others = data['Other Candidates']
            if self.winning_party == 'Democratic':
                self.president = self.democratic
            elif self.winning_party == 'Republican':
                self.president = self.republican
            elif self.winning_party == 'Democratic-Republican':
                self.president = self.dem_rep            
            else:
                self.president = self.others.split('/')[0].split(' (')[0]             
            US_ELECTION.president_list[self.year] = self

class US_PARTY():
    
    party_list = {}
    
    def __init__(self,data):
        self.name = data['Party']
        self.colour = data['Colour']
        self.colour_scale = data['Colour Scale'].split(',')
        self.coalition = data['Main Party']
        US_PARTY.party_list[self.name] = self
    
    def get_colour(party):
        if party in US_PARTY.party_list.keys():
            return US_PARTY.party_list[party].colour
        else:
            return '#C8C7C9'
        
def process_house(data):
    
    data['Party Colour'] = data['Party'].apply(lambda x: US_PARTY.get_colour(x))
    data['Percent'] = data['Percent'].apply(lambda x: '{:.1f}'.format(x) if x != '' else x)
    
    for state in STATE.state_list.keys():
        if state in list(data['State']):
            for district in data[data['State'] == state]['District'].unique():
                res = data[(data['State']==state) & (data['District']==district)]
                STATE.state_list[state].district_list[district].add_results(res)

def process_senate(data):
    
    years = list(set(data['Year']))
    data['Party Colour'] = data['Party'].apply(lambda x: US_PARTY.get_colour(x))
    data['Percent'] = data['Percent'].apply(lambda x: '{:.1f}'.format(x) if x != '' else x)
    
    for year in years:
        norm_res = data[(data['Year'] == year) & (data['Special'] == '')]
        spec_res = data[(data['Year'] == year) & (data['Special'] != '')]
        
        for state in STATE.state_list.keys():
            if STATE.state_list[state].name != 'District of Columbia':
                obj = STATE.state_list[state]
                if state in list(norm_res['State']):
                    obj.process_senate_results(norm_res[norm_res['State'] == state],str(year))    
                if state in list(spec_res['State']):
                    obj.process_senate_results(spec_res[spec_res['State'] == state],str(year),special=True)

def process_pres(data):
    
    data['Party Colour'] = data['Party'].apply(lambda x: US_PARTY.get_colour(x))
    #data['Percent'] = data['Percent'].apply(lambda x: '{:.1f}'.format(x) if x != '' else x)
    
    years = list(set(data['Year']))
    
    for year in years:
        
        for state in STATE.state_list.keys():
            obj = STATE.state_list[state]
            obj.process_pres_results(data[data['State'] == state],str(year))


In [38]:
# Load US data from Excel

us_data_list = [("States",STATE),("Elections",US_ELECTION),("Dist_sum",DISTRICT),("Parties",US_PARTY)]

for data_type in us_data_list:
    df = pd.read_excel("US_Elections.xlsx",sheet_name=data_type[0])
    df.fillna('',axis=1,inplace=True)
    for row in df.index:
        data_type[1](df.loc[row]) 

dist_full_df = pd.read_excel("US_Elections.xlsx",sheet_name="Dist_full")
dist_full_df.fillna('',axis=1,inplace=True)
process_house(dist_full_df)

sen_full_df = pd.read_excel("US_Elections.xlsx",sheet_name="Sen_full")
sen_full_df.fillna('',axis=1,inplace=True)
process_senate(sen_full_df)

pres_full_df = pd.read_excel("US_Elections.xlsx",sheet_name="Pres_full")
pres_full_df.fillna('',axis=1,inplace=True)
process_pres(pres_full_df)

us_input_dict = [{'filename':'SVGs/50States.svg','elections':['2020']}]

for group in us_input_dict:
    xs, ys, names = convert_svg(group['filename'],'US',group['elections'][0])
    for election in group['elections']:
        US_ELECTION.svg_data[election] = {'xs':xs,'ys':ys,'names':names}


In [39]:
# List of colours for counties
county_colours = {"Essex":"orange",
"Bedfordshire":"green",
"Norfolk":"black",
"Hertfordshire":"pink",
"Suffolk":"blue",
"Cambridgeshire":"purple",
"Derbyshire":"blue",
"Nottinghamshire":"green",
"Lincolnshire":"pink",
"Leicestershire":"yellow",
"Northamptonshire":"lightgreen",
"London":"red",
"Northumberland":"red",
"Durham":"blue",
"Tyne and Wear":"pink",
"Cleveland":"green",
"Greater Manchester":"green",
"Cumbria":"orange",
"Merseyside":"lightgreen",
"Lancashire":"purple",
"Cheshire":"yellow",
"Northern Ireland":"red",
"Scotland":"green",
"Hampshire":"purple",
"West Sussex":"yellow",
"Kent":"blue",
"Buckinghamshire":"black",
"Oxfordshire":"blue",
"East Sussex":"green",
"Berkshire":"lightgreen",
"Surrey":"orange",
"Isle of Wight":"red",
"Wiltshire":"blue",
"Avon":"red",
"Dorset":"green",
"Somerset":"yellow",
"Cornwall":"blue",
"Devon":"orange",
"Gloucestershire":"purple",
"Wales":"pink",
"West Midlands":"red",
"Worcestershire":"green",
"Staffordshire":"lightblue",
"Herefordshire":"yellow",
"Warwickshire":"orange",
"Shropshire":"black",
"South Yorkshire":"orange",
"West Yorkshire":"yellow",
"Humberside":"purple",
"North Yorkshire":"red",}


In [41]:
# Main GUI
# Assumptions: countries hard-coded, initial load hard-coded, map option always available compared to hex option
# Map: centre_point attribute in election class, 'name' field in geojson, map_list and hex_list in election class

class GUI():
    
    def __init__(self):
        
        self.country = 'UK'
        self.election = '2019'
        self.ECLASS = ELECTION
        self.DCLASS = CONSTITUENCY
        self.PCLASS = PARTY
        self.view = 'Map'
        self.hex_country_list = ['UK','US']
        self.us_election = 'House'
        self.title = widgets.HTML("<h2>United Kingdom General Elections</h2>")
        self.country_option = widgets.ToggleButtons(options=['UK','US'],
                                                layout=Layout(margin="10px 0px 20px 0px"),
                                                button_style='danger')
        self.election_option = widgets.ToggleButtons(options=sorted(list(set(self.ECLASS.map_list + self.ECLASS.hex_list)),reverse=True),
                                        layout=Layout(margin="10px 0px 20px 0px"),
                                        button_style='warning')
        self.us_option = widgets.ToggleButtons(options=['House','Senate','President'],
                                        layout=Layout(margin="10px 0px 20px 0px"),
                                        button_style='success')
        self.view_option = widgets.ToggleButtons(options=['Map','Hex'],button_style='info')
        self.country_option.observe(self.switch_country,names='value')
        self.election_option.observe(self.switch_election,names='value')
        self.us_option.observe(self.switch_us,names='value')
        self.view_option.observe(self.switch_view,names='value')
        self.override_switch = False
        
        self.over_label = widgets.HTML("<b>Constituency at cursor:</b>")
        self.info_box_over = widgets.Text()
        self.info_box_click = widgets.Output()
        self.info_box = widgets.VBox([self.over_label,self.info_box_over,self.info_box_click],layout=Layout(width="40%"))
        
        self.map_output = widgets.VBox()
        self.hex_output = widgets.Output()
        self.svg_output = widgets.Output()
        
        self.hex_box = widgets.VBox([self.view_option,self.hex_output],layout=Layout(width='100%'))
        self.map_box = widgets.VBox([self.view_option,self.map_output],layout=Layout(width='65%',height='900px'))
        self.svg_box = widgets.VBox([self.view_option,self.svg_output],layout=Layout(width='100%'))
        
        self.map_info_box = widgets.HBox([self.map_box,self.info_box],layout=Layout(height="900px",width="100%"))
        
        self.menu_options = widgets.ToggleButtons(options=['Elections','Constituencies'], button_style='success')
        self.menu_options.observe(self.switch_detail,names='value')
        self.detail_list = widgets.Dropdown(options=ELECTION.elect_list.keys())
        self.detail_list.observe(self.load_detail,names='value')
        self.menu_bar = widgets.HBox([self.menu_options,self.detail_list])
        self.detail_output = widgets.Output()
        self.detail_box = widgets.VBox([self.menu_bar,self.detail_output],layout=Layout(width="100%"))
        
        self.gui = widgets.VBox([self.title,self.country_option,self.election_option,self.map_info_box,self.detail_box],layout=Layout(align_items='center'))
        
        #self.load_view()
        self.load_detail('')
    
    def change_title(new_title):
        
        self.title.value = "<h2>" + str(new_title) + "<\h2>"

    def get_inputs(self):
        
        self.country = self.country_option.value
        self.election = self.election_option.value
        if self.country == 'UK':
            self.ECLASS = ELECTION
            self.DCLASS = CONSTITUENCY
            self.PCLASS = PARTY
        elif self.country == 'US':
            self.ECLASS = US_ELECTION
            self.PCLASS = US_PARTY
            self.us_election = self.us_option.value
            if self.us_election == 'House':
                self.DCLASS = DISTRICT
            else:
                self.DCLASS = STATE
        self.view = self.view_option.value
    
    def switch_country(self,wid):
        
        self.override_switch = True
        self.get_inputs()
        
        self.election_option.options = sorted(list(set(self.ECLASS.map_list + self.ECLASS.hex_list)),reverse=True)
        
        if self.country not in self.hex_country_list:
            self.view_option.options = ['Map']
        else:
            self.view_option.options = ['Map','Hex']
        
        if self.country == 'US':
            self.gui.children = [self.title,self.country_option,self.election_option,self.us_option,self.map_info_box]
        else:
            self.gui.children = [self.title,self.country_option,self.election_option,self.map_info_box]
        
        self.override_switch = False
        self.get_inputs()
        self.load_view()
    
    def switch_election(self,wid):
        
        if self.override_switch:
            return
        
        self.override_switch = True
        
        self.get_inputs()
        
        if self.view == 'Map':
            if self.country in self.hex_country_list and self.election in self.ECLASS.hex_list:
                self.view_option.options = ['Map','Hex']
            else:
                self.view_option.options = ['Map']
        elif self.view == 'Hex':
            self.view_option.options = ['Map','Hex']
        
        if self.country == 'US':
            if self.election in self.ECLASS.president_list:
                self.us_option.options = ['House','Senate','President']
            else:
                self.us_option.options = ['House','Senate']
        
        self.override_switch = False
        
        self.load_view()
    
    def switch_us(self,wid):

        if self.override_switch:
            return
        
        self.override_switch = True
        
        self.get_inputs()
        
#         if self.us_election == 'House':
#             self.view_option.options = ['Map','Hex']
#             if self.view == 'Hex':
#                 self.view_option.value = 'Hex'
#         else:
#             self.view_option.options = ['Hex']
        
        if self.us_election == 'President':
            if self.view == 'Map':
                pres_years = set(self.ECLASS.map_list).intersection(set(self.ECLASS.president_list))
                self.election_option.options = sorted(list(pres_years),reverse=True)
            elif self.view == 'Hex':
                pres_years = set(self.ECLASS.hex_list).intersection(set(self.ECLASS.president_list))
                self.election_option.options = sorted(list(pres_years),reverse=True)
        
        self.get_inputs()
        
        self.override_switch = False
        
        self.load_view()
    
    def switch_view(self,wid):
        
        if self.override_switch:
            return
        
        self.override_switch = True
        
        self.get_inputs()
        
        if self.view_option.value == 'Map':
            self.election_option.options = sorted(list(self.ECLASS.map_list),reverse=True)
        if self.view_option.value == 'Hex':
            self.election_option.options = sorted(list(self.ECLASS.hex_list),reverse=True)
        
        self.override_switch = False
        
        self.load_view()
    
    def switch_detail(self,wid):
        
        detail_type = self.menu_options.value
        
        if detail_type == 'Elections':
            self.detail_list.options = ELECTION.elect_list.keys()
        elif detail_type == 'Constituencies':
            self.detail_list.options = sorted(list(CONSTITUENCY.const_list.keys()))
    
    def load_detail(self,wid):
        
        detail_type = self.menu_options.value
        
        self.detail_output.clear_output()
        
        html = '''<style>
            table {
              font-family: arial, sans-serif;
              border-collapse: collapse;
              width: 60%;
            }

            td {
              border: 0.5px solid #000000;
              text-align: left;
              padding: 1px 5px 1px 5px;
            }

            th {
              border: 0.5px solid #000000;
              background-color: #E7E6E7;
              text-align: center;
              padding: 1px 5px 1px 5px; 
            }

            .blank {
              border: none;
            }
            </style>''' 
        
        with self.detail_output:
            
            if detail_type == 'Elections':
                
                election = self.detail_list.value
                obj = ELECTION.elect_list[election]
                df = obj.result_df
                
                if obj.end_date == '':
                    html += '<div><b>Date: </b>' + str(int(obj.start_date.strftime('%d'))) + \
                            obj.start_date.strftime(' %b %Y') + '</div>'
                else:
                    html += '<div><b>Date: </b>' + str(int(obj.start_date.strftime('%d'))) + \
                            obj.start_date.strftime(' %b %Y') + ' - ' + str(int(obj.end_date.strftime('%d'))) + \
                            obj.end_date.strftime(' %b %Y') + '</div>'
                html += '<div><b>Largest Party: </b>' + obj.largest_party + '</div>'
                html += '<div><b>Prime Minister: </b>' + obj.pm + '</div>'
                #html += '<div><b>Turnout: </b>' + obj.turnout + '</div>'
                
                seats = 0
                res = ELECTION.elect_list[election].result_df
                for row in res.index:
                    seats += res.Seats[row]
                html += '<div><b>Total Seats: </b>' + str(seats) + '</div>'
                html += '<div><b>Total Votes: </b>' + '{:,}'.format(obj.total_votes) + '</div>'
                
                html += '<table><tr><th colspan="2">Party</th><th>Seats</th><th>Votes</th><th>Percent</th></tr>'
                
                for row in df.index:
                    party = PARTY.party_list.get(df.Party[row],'')
                    if party == '':
                        party_colour = '#C8C7C9'
                    else:
                        party_colour = party.colour
                    html += '<tr><td style="width:4px;background-color:' + party_colour + ';">  </td>'
                    html += '<td>' + df.Party[row] + '</td>'
                    html += '<td style="text-align:center;">' + str(df.Seats[row]) + '</td>'
                    html += '<td style="text-align:center;">' + '{:,}'.format(df.Votes[row]) + '</td>'
                    html += '<td style="text-align:center;">' + '{:.1f}'.format(df.Percent[row]) + '</td></tr>'
                
            elif detail_type == 'Constituencies':
            
                name = self.detail_list.value
                obj = CONSTITUENCY.const_list[name]                     
                
                html += '<div><b>Region: </b>' + obj.region + '</div>'
                html += '<div><b>County: </b>' + obj.county + '</div>'
                html += '<div><b>Created: </b>' + obj.created.strftime('%Y')
                if obj.predecessors1 == '':
                    html += '</div>'
                else:
                    html += '&nbsp&nbsp&nbsp&nbsp<b>Predecessors: </b>' + obj.predecessors1 + '</div>'
                if obj.abolished != []:
                    for i in range(1,len(obj.abolished)+1):
                        html += '<div><b>Abolished: </b>' + obj.abolished[i-1].strftime('%Y')
                        html += '&nbsp&nbsp&nbsp&nbsp<b>Successors: </b>' + obj.successors.split('|')[i-1] + '</div>'
                        if len(obj.recreated) >= i:
                            html += '<div><b>Recreated: </b>' + obj.recreated[i-1].strftime('%Y')
                            html += '&nbsp&nbsp&nbsp&nbsp<b>Predecessors: </b>' + obj.predecessors2.split('|')[i-1]
                            html += '</div>'
                
                if obj.previous != '':
                    for i in range(0,len(obj.previous.split('|'))):
                        html += '<div><b>Previous Name: </b>' + obj.previous.split('|')[i]
                        html += '<b>Date changed: </b>' + obj.changed[i].strftime('%Y') + '</div>'
                if obj.singleMP != []:
                    html += '<div>Represented by one MP from ' + obj.singleMP[0].strftime('%Y') + ' until '
                    html += obj.singleMP[1].strftime('%Y') + '</div>'
                if obj.doubleMP != []:
                    html += '<div>Represented by two MPs from ' + obj.doubleMP[0].strftime('%Y') + ' until '
                    html += obj.doubleMP[1].strftime('%Y') + '</div>'
                if obj.tripleMP != []:
                    html += '<div>Represented by three MPs from ' + obj.tripleMP[0].strftime('%Y') + ' until '
                    html += obj.tripleMP[1].strftime('%Y') + '</div>'
                
                for election in sorted(list(obj.election_list.keys()), reverse=True):
                    if election[-1] == 'B':
                        if election[-2] == 'B':
                            byelection = election[:-2]
                        else:
                            byelection = election[:-1]
                        html += '<h3 style="text-align:left;">' + byelection + ' By-Election Result</h3>'
                    else:
                        html += '<h3 style="text-align:left;">' + election + ' General Election Result</h3>'
                    
                    res_obj = obj.election_list[election]
                    df = res_obj.results.copy()
                    notes = ''
                    html += '<table><tr><th colspan="2">Party</th><th>Candidate</th><th>Votes</th><th>Percent</th></tr>'
                    count = 1
                    for row in df.index:
                        party = PARTY.party_list.get(df.Party[row],'')
                        if party == '':
                            party_colour = '#C8C7C9'
                        else:
                            party_colour = party.colour
                        if count <= res_obj.seats:
                            html += '<tr style="font-weight: bold;">'
                        else:
                            html += '<tr>'
                        html += '<td style="width:4px;background-color:' + party_colour + ';">  </td>'
                        html += '<td>' + df.Party[row] + '</td>'
                        html += '<td>' + df.Candidate[row] + '</td>'
                        if df.Votes[row] == 'Unopposed':
                            html += '<td colspan=2 style="text-align:center;">Unopposed</td></tr>'
                        else:
                            html += '<td style="text-align:center;">' + '{:,}'.format(df.Votes[row]) + '</td>'
                            html += '<td style="text-align:center;">' + '{:.1f}'.format(df.Percent[row]) + '</td></tr>'
                            if df.Notes[row] != '':
                                notes += '<div>' + df.Notes[row] + '</div>'
                        count += 1
                    if not res_obj.unopposed:
                        majority_percent = res_obj.majority / res_obj.turnoutN * 100
                        if res_obj.seats == 1:
                            html += '<tr><td colspan="2" class="blank"></td><th>Majority</th>'
                            html += '<td style="text-align:center;">' + '{:,}'.format(res_obj.majority) + '</td>'
                            html += '<td style="text-align:center;">' + '{:.1f}'.format(majority_percent) + '</td></tr>'
                        html += '<tr><td colspan="2" class="blank"></td><th>Turnout</th>'
                        html += '<td style="text-align:center;">' + '{:,}'.format(res_obj.turnoutN) + '</td>'
                        if isinstance(res_obj.turnoutP,str):
                            html += '<td style="text-align:center;">' + res_obj.turnoutP + '</td></tr>'
                        else:
                            html += '<td style="text-align:center;">' + '{:.1f}'.format(res_obj.turnoutP) + '</td></tr>'
                        html += '</table>'
                    else:
                        html += '</table>'
                    if res_obj.notes != '':
                        if res_obj.notes[0] == '[':
                            notes += '<div>' + res_obj.notes.split(']')[1] + '</div>'
                        else:
                            notes += '<div>' + res_obj.notes + '</div>'
                    html += notes
        
            display(widgets.HTML(html))
        
    def load_view(self):
        
        if self.view == 'Map':
            if self.ECLASS.elect_list[self.election].map != 'SVG':
                if self.country == 'US' and self.us_election != 'House':
                    self.map_info_box.children = [self.svg_box]
                    self.load_svg()
                else:
                    self.map_info_box.children = [self.map_box,self.info_box]
                    self.load_map()
            else:
                self.map_info_box.children = [self.svg_box]
                self.load_svg()

        elif self.view == 'Hex':
            self.map_info_box.children = [self.hex_box]
            self.load_hex()

    def load_hex(self):
        
        def get_hex_coords(hex_coords):
    
            final_coords = []
            for hex_string in hex_coords:
                hex_split = hex_string.split(',')
                hex_split = [int(x) for x in hex_split]
                final_coords.append(hex_split)

            # Horizontal cartesian coords
            hcoord_n = [c[0] for c in final_coords]
            # Vertical cartersian coords
            vcoord_n = [2. * np.sin(np.radians(60)) * (c[1] - c[2]) /3. for c in final_coords]

            x_list = []
            y_list = []
            for i in range(0,len(hcoord_n)):
                hex1 = RegularPolygon((hcoord_n[i], vcoord_n[i]), numVertices=6, radius=2. / 3.,orientation=np.radians(30))
                points = hex1.get_verts().tolist()
                x_list.append([m[0] for m in points])
                y_list.append([m[1] for m in points])        

            return x_list, y_list
        
        def get_colours_and_results():
            
            if self.country == 'US' and (self.us_election == 'Senate' or self.us_election == 'President'):
                results = []
                colours = []
                hex_aspect = US_ELECTION.state_hex_aspect
                for state in names:
                    if self.us_election == 'President':
                        res = STATE.state_list[state].pres_results[self.election]
                        results.append(res.results.to_dict())
                        colours.append(US_PARTY.party_list[res.party].colour)
                    elif self.us_election == 'Senate' and state != 'District of Columbia':
                        res = STATE.state_list[state].senate_results
                        if self.election not in res.keys():
                            results.append({})
                            colours.append('#C8C7C9')
                        else:
                            no_ord = True
                            for contest in res[self.election]:
                                if not contest.special:
                                    results.append(contest.results.to_dict())
                                    colours.append(US_PARTY.party_list[contest.party].colour)
                                    no_ord = False
                            if no_ord:
                                results.append({})
                                colours.append('#C8C7C9')
            else:
                
                if self.country == 'US' and self.us_election == 'House':
                    hex_aspect = US_ELECTION.house_hex_aspect
                else:
                    hex_aspect = self.ECLASS.hex_aspect
                
                results = [self.DCLASS.const_list[x].election_list[self.election].results.to_dict() for x in names]
                colours = self.get_colours(names,self.election,'party')
            
            return colours, results, hex_aspect
        
        self.hex_output.clear_output()
        
        if self.country == 'US':
            if self.us_election == 'Senate':
                df = self.ECLASS.state_hex_df
                hex_col = self.ECLASS.elect_list[self.election].state_hex
            elif self.us_election == 'President':
                df = self.ECLASS.state_hex_df
                hex_col = self.ECLASS.elect_list[self.election].pres_hex
            elif self.us_election == 'House':
                df = self.ECLASS.house_hex_df
                hex_col = self.ECLASS.elect_list[self.election].house_hex
        else:
            df = self.ECLASS.hex_df
            hex_col = self.ECLASS.elect_list[self.election].hex
        
        df = df[df[hex_col] != ""]
        df = df[df[hex_col] != "Yes"]
        
        if self.country == 'US':
            if self.us_election == 'House':
                div_name = 'District'
            else:
                div_name = 'State'
        else:
            div_name = self.ECLASS.division_name
        
        with self.hex_output: 
            
            names = list(df[div_name])
            coords = list(df[hex_col])
            xs, ys = get_hex_coords(coords)
            colours, results, hex_aspect = get_colours_and_results()
            
            count = 0
            for const in results:
                for key in const.keys():
                    for subkey in const[key].keys():
                        results[count][key][subkey] = str(results[count][key][subkey])
                count += 1
            
            data=dict(x=xs, y=ys, name=names, colours=colours, results = results)

            cds = ColumnDataSource(data)

            TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"
            
            #title="General Election " + self.election
            p = figure(tools=TOOLS, x_axis_location=None, y_axis_location=None, tooltips=[("Name", "@name")],aspect_ratio=hex_aspect)
            
            p.grid.grid_line_color = None
            p.hover.point_policy = "follow_mouse"

            patch_renderer  = p.patches('x', 'y', source=cds,
                      fill_color={"field":"colours"},
                      fill_alpha=0.7, line_color="white", line_width=0.5)
            
            indicator_div = Div(text="",min_width=700)
            layout = bkCol(bkRow(p, indicator_div),width_policy="max")

            tap_tool = TapTool(renderers=[patch_renderer])
            p.add_tools(tap_tool)
            
            patch_indicator_callback = CustomJS(args=dict(cds=cds, div=indicator_div, election=self.election), code=self.get_bokeh_code())

            cds.selected.js_on_change('indices', patch_indicator_callback)

            show(layout)

    def load_map(self):
        
        def get_party_colour(feature):
            
            name = feature['properties']['name']
            
            if name in self.DCLASS.const_list.keys():
                obj = self.DCLASS.const_list[name]
            elif hasattr(self.DCLASS,'previous_list') and name in self.DCLASS.previous_list.keys():
                obj = self.DCLASS.previous_list[name]
            else:
                return {'fillColor':'#FFFFFF',
                        'color':'#000000',
                        'weight':1,
                        'fillOpacity':0.8}
            
            if self.country == 'UK':
                party = obj.election_list[self.election].parties[0]
            else:
                party = obj.election_list[self.election].party1
            if party in self.PCLASS.party_list.keys():
                colour = self.PCLASS.party_list[party].colour
            else:
                colour = '#FFFFFF'
            
            return {'fillColor':colour,
                    'color':'#000000',
                    'weight':1,
                    'fillOpacity':0.8}
        
        self.map_output.children = [widgets.HTML("<br><br><br>Loading...")]
        
        with open('Data/' + self.country + '/' + self.ECLASS.elect_list[self.election].map) as f:
            gj = geojson.load(f)
        
        geo_json = GeoJSON(
            data=gj,
            style={'opacity': 1, 'fillOpacity': 0.2, 'weight': 1},
            hover_style={'color': 'black', 'dashArray': '0', 'fillOpacity': 0.5},
            style_callback=get_party_colour
        )

        geo_json.on_hover(self.hover_func)
        geo_json.on_click(self.click_func)    
        
        m = Map(center=self.ECLASS.centre_point, zoom=5, scroll_wheel_zoom=True, layout=Layout(height="700px"))
        m.add_layer(geo_json)
        
        self.map_output.children = [m]

    def load_svg(self):
        
        self.svg_output.clear_output()
        
        with self.svg_output:
            
            load = widgets.HTML("<br><br><br>Loading...")
            display(load)
            
            svg_dict = self.ECLASS.svg_data[self.election]
            
            names = svg_dict['names']
            
            if 'test' in svg_dict and svg_dict['test']:
                colours = ['#5F5763'] * len(names)
                results = [''] * len(names)
            else:
                colours = self.get_colours(names, self.election)
                results = self.get_results(names)
            
            data = dict(x=svg_dict['xs'],y=svg_dict['ys'],name=names,colours=colours,results=results)
            
            cds = ColumnDataSource(data)

            TOOLS = "pan,wheel_zoom,reset,hover,save"

            p = figure(title="General Election " + self.election, tools=TOOLS, tooltips=[("Name", "@name")],
                x_axis_location=None, y_axis_location=None, aspect_ratio=self.ECLASS.svg_aspect)

            patch_renderer = p.multi_polygons(xs="x",ys="y",line_width=1,color="colours",line_color='black',
                                              name="names",source=cds)
            
            p.hover.point_policy = "follow_mouse"

            indicator_div = Div(text="",min_width=700)
            layout = bkCol(bkRow(p, indicator_div),width_policy="max")
            
            tap_tool = TapTool(renderers=[patch_renderer])
            p.add_tools(tap_tool)
            display_text = self.get_bokeh_code()
            patch_indicator_callback = CustomJS(args=dict(cds=cds, div=indicator_div, election=self.election), 
                                                code=display_text)
            
            cds.selected.js_on_change('indices', patch_indicator_callback)
            
            load.close()
            
            show(layout)
    
    def get_colours(self, consts, election, mode='party'):

        colours = []
        default_colour_scale = ['#403943','#5F5763','#7F7484','#A199A5','#C4BEC6']
        
        #if election == '2020':
            #return ['#7F7484'] * len(consts)
        
        if mode[:2] == 'SP':
            
            party = mode[2:]
            coalition = ''
            if PARTY.party_list[party].coalition != '':
                coalition = PARTY.party_list[party].coalition
            
            if party in PARTY.party_list and PARTY.party_list[party].colour_scale != []:
                colour_scale = PARTY.party_list[party].colour_scale
            else:
                colour_scale = ['#403943','#5F5763','#7F7484','#A199A5','#C4BEC6']
              
            percents = []
            
            for const in consts:
                if const not in CONSTITUENCY.const_list.keys():
                    try:
                        const_obj = CONSTITUENCY.previous_list[const]
                    except:
                        raise Exception('Constituency not in database',const,election)
                else:
                    const_obj = CONSTITUENCY.const_list[const]
                df = const_obj.election_list[election].results
                if party in list(df.Party):
                    percent_series = df.loc[df['Party'] == party, 'Percent']
                    if percent_series[percent_series.index[0]] != '':
                        percents.append(percent_series[percent_series.index[0]])
            
            arrays = np.array_split(sorted(percents,reverse=True),5)
            
            const_count = 0
            colour_count = 0
            for const in consts:
                if const not in CONSTITUENCY.const_list.keys():
                    try:
                        const_obj = CONSTITUENCY.previous_list[const]
                    except:
                        raise Exception('Constituency not in database',const,election)
                else:
                    const_obj = CONSTITUENCY.const_list[const]
                df = const_obj.election_list[election].results
                if party in list(df.Party):
                    percent_series = df.loc[df['Party'] == party, 'Percent']
                    percent = percent_series[percent_series.index[0]]
                    count = 0
                    for array in arrays:
                        if percent in array:
                            colour_count += 1
                            colours.append(colour_scale[count])
                            break
                        count += 1
                else:
                    colours.append('#C3C4BE')
                const_count += 1
                #print(const_count,colour_count,end=':')
                #if colour_count > 650:
                    #print(const)
            
            return colours
        
        winners = self.DCLASS.get_winners(consts, election, election_type=self.us_election)
        
        colours = [self.PCLASS.party_list[party].colour for party in winners]
        
#         for const in consts:

#             if const not in self.DCLASS.const_list.keys():
#                 const_obj = self.DCLASS.previous_list[const]
#             else:
#                 const_obj = self.DCLASS.const_list[const]

#             if mode == 'party':

#                 try:
#                     results = const_obj.election_list[election]
#                 except:
#                     raise Exception('Election results not found',const,election)

#                 winner = results.party1

#                 try:
#                     colours.append(self.PCLASS.party_list[winner].colour)
#                 except:
#                     raise Exception('Winning party not found',winner)
                
#             elif mode == 'county':

#                 county = const_obj.county
#                 colours.append(county_colours.get(county,'pink'))

        return colours

    def get_results(self, names):
        
        results = []
        
        if self.country == 'UK':
            for name in names:
                if name in CONSTITUENCY.const_list.keys():
                    results.append(CONSTITUENCY.const_list[name].election_list[self.election].results.to_dict())
                else:
                    results.append(CONSTITUENCY.previous_list[name].election_list[self.election].results.to_dict())
        else:
            for name in names:
                try:
                    results.append(STATE.state_list[name].senate_results[self.election][0].results.to_dict())
                except:
                    results.append('No Election')
        
        return results

    def get_bokeh_code(self):
        
        display_text = ""
        
        if self.country == 'UK':
            #"td {border: 0.5px solid #000000;text-align: left;padding: 1px 5px 1px 5px;width: 0.1%;white-space: nowrap;}" +
            display_text = """div.text = "<style>table {font-family: arial, sans-serif;border-collapse: collapse;width: 60%;}" +
                    "td {border: 0.5px solid #000000;text-align: left;padding: 1px 5px 1px 5px;}" +
                    "th {border: 0.5px solid #000000;background-color: #E7E6E7;text-align: center;padding: 1px 5px 1px 5px;}" +
                    ".blank {border: none;}</style>" +
                    "<h2>" + cds.data['name'][cb_obj.indices] + "</h2><h3>" + election + " General Election Results</h3><br>" + 
                      "<b>Winning Party:</b> " + cds.data['results'][cb_obj.indices]['Party'][0] + "<br><br>" +
                      "<table>" +
                      "<tr>" +
                        "<th style='width:1px;'> </th>" + 
                        "<th>Party</th>" +
                        "<th>Candidate</th>" +
                        "<th style='width:60px;'>Votes</th>" +
                        "<th style='width:60px;'>Percent</th>" +
                      "</tr>"
                      let partyList = cds.data['results'][cb_obj.indices]['Party']
                      for (let i = 0; i < Object.keys(partyList).length; i++) {
                        div.text += "<tr><td style='width:1px;background-color:" + cds.data['results'][cb_obj.indices]['Party Colour'][i] + ";'> </td>";
                        div.text += "<td>" + cds.data['results'][cb_obj.indices]['Party'][i] + "</td>";
                        div.text += "<td>" + cds.data['results'][cb_obj.indices]['Candidate'][i] + "</td>";
                        div.text += "<td style='width:60px;'>" + cds.data['results'][cb_obj.indices]['Votes'][i] + "</td>";
                        div.text += "<td style='width:60px;'>" + cds.data['results'][cb_obj.indices]['Percent'][i] + "</td></tr>";
                        }
                      div.text += "</table>"
                   """
        elif self.country == 'US':
            #"td {border: 0.5px solid #000000;text-align: left;padding: 1px 5px 1px 5px;width: 0.1%;white-space: nowrap;}" +
            display_text = """div.text = "<style>table {font-family: arial, sans-serif;border-collapse: collapse;width: 60%;}" +
                    "td {border: 0.5px solid #000000;text-align: left;padding: 1px 5px 1px 5px;}" +
                    "th {border: 0.5px solid #000000;background-color: #E7E6E7;text-align: center;padding: 1px 5px 1px 5px;}" +
                    ".blank {border: none;}</style>" +
                    "<h2>" + cds.data['name'][cb_obj.indices] + "</h2><h3>" + election + " General Election Results</h3><br>" + 
                      "<b>Winning Party:</b> " + cds.data['results'][cb_obj.indices]['Party'][0] + "<br><br>" +
                      "<table>" +
                      "<tr>" +
                        "<th style='width:1px;'> </th>" + 
                        "<th>Party</th>" +
                        "<th>Candidate</th>" +
                        "<th style='width:60px;'>Votes</th>" +
                        "<th style='width:60px;'>Percent</th>" +
                      "</tr>"
                      let partyList = cds.data['results'][cb_obj.indices]['Party']
                      for (let i = 0; i < Object.keys(partyList).length; i++) {
                        div.text += "<tr><td style='width:1px;background-color:" + cds.data['results'][cb_obj.indices]['Party Colour'][i] + ";'> </td>";
                        div.text += "<td>" + cds.data['results'][cb_obj.indices]['Party'][i] + "</td>";
                        div.text += "<td>" + cds.data['results'][cb_obj.indices]['Candidate'][i] + "</td>";
                        div.text += "<td style='width:60px;'>" + cds.data['results'][cb_obj.indices]['Votes'][i] + "</td>";
                        div.text += "<td style='width:60px;'>" + cds.data['results'][cb_obj.indices]['Percent'][i] + "</td></tr>";
                        }
                      div.text += "</table>"
                   """
        
        return display_text
    
    def hover_func(self,event,feature,properties):
        
        self.info_box_over.value = str(properties['name'])

    def click_func(self,event,feature,properties):

        self.info_box_click.clear_output()

        with self.info_box_click:
            
            if self.country == 'UK':
                
                name = properties['name']
                obj = CONSTITUENCY.const_list[name]
                election = self.election_option.value
                html = '''<style>
                        table {
                          font-family: arial, sans-serif;
                          border-collapse: collapse;
                          width: 60%;
                        }

                        td {
                          border: 0.5px solid #000000;
                          text-align: left;
                          padding: 1px 5px 1px 5px;
                        }

                        th {
                          border: 0.5px solid #000000;
                          background-color: #E7E6E7;
                          text-align: center;
                          padding: 1px 5px 1px 5px; 
                        }

                        .blank {
                          border: none;
                        }
                        </style>'''                        

                html += '<h2>' + name + '</h2>'
                html += '<div><b>Region: </b>' + obj.region + '</div>'
                html += '<div><b>County: </b>' + obj.county + '</div>'
                html += '<div><b>Created: </b>' + obj.created.strftime('%Y')
                if obj.predecessors1 == '':
                    html += '</div>'
                else:
                    html += '&nbsp&nbsp&nbsp&nbsp<b>Predecessors: </b>' + obj.predecessors1 + '</div>'
                if obj.abolished != []:
                    for i in range(1,len(obj.abolished)+1):
                        html += '<div><b>Abolished: </b>' + obj.abolished[i-1].strftime('%Y')
                        html += '&nbsp&nbsp&nbsp&nbsp<b>Successors: </b>' + obj.successors.split('|')[i-1] + '</div>'
                        if len(obj.recreated) >= i:
                            html += '<div><b>Recreated: </b>' + obj.recreated[i-1].strftime('%Y')
                            html += '&nbsp&nbsp&nbsp&nbsp<b>Predecessors: </b>' + obj.predecessors2.split('|')[i-1]
                            html += '</div>'

                if obj.previous != '':
                    for i in range(0,len(obj.previous.split('|'))):
                        html += '<div><b>Previous Name: </b>' + obj.previous.split('|')[i]
                        html += '&nbsp&nbsp&nbsp&nbsp<b>Date changed: </b>' + obj.changed[i].strftime('%Y') + '</div>'
                if obj.singleMP != []:
                    html += '<div>Represented by one MP from ' + obj.singleMP[0].strftime('%Y') + ' until '
                    html += obj.singleMP[1].strftime('%Y') + '</div>'
                if obj.doubleMP != []:
                    html += '<div>Represented by two MPs from ' + obj.doubleMP[0].strftime('%Y') + ' until '
                    html += obj.doubleMP[1].strftime('%Y') + '</div>'
                if obj.tripleMP != []:
                    html += '<div>Represented by three MPs from ' + obj.tripleMP[0].strftime('%Y') + ' until '
                    html += obj.tripleMP[1].strftime('%Y') + '</div>'

                html += '<h3 style="text-align:left;">' + election + ' General Election Results</h3>'

                res_obj = obj.election_list[election]
                df = res_obj.results.copy()
                notes = ''
                html += '<table><tr><th colspan="2">Party</th><th>Candidate</th><th>Votes</th><th>Percent</th></tr>'

                for row in df.index:
                    party = PARTY.party_list.get(df.Party[row],'')
                    if party == '':
                        party_colour = '#C8C7C9'
                    else:
                        party_colour = party.colour
                    html += '<tr><td style="background-color:' + party_colour + ';">  </td>'
                    html += '<td>' + df.Party[row] + '</td>'
                    html += '<td>' + df.Candidate[row] + '</td>'
                    if df.Votes[row] == 'Unopposed':
                        html += '<td colspan=2 style="text-align:center;">Unopposed</td></tr>'
                    else:
                        html += '<td style="text-align:center;">' + '{:,}'.format(df.Votes[row]) + '</td>'
                        html += '<td style="text-align:center;">' + '{:.1f}'.format(df.Percent[row]) + '</td></tr>'
                        if df.Notes[row] != '':
                            notes += '<div>' + df.Notes[row] + '</div>'
                if not res_obj.unopposed:
                    majority_percent = res_obj.majority / res_obj.turnoutN * 100
                    html += '<tr><td colspan="2" class="blank"></td><th>Majority</th>'
                    html += '<td style="text-align:center;">' + '{:,}'.format(res_obj.majority) + '</td>'
                    html += '<td style="text-align:center;">' + '{:.1f}'.format(majority_percent) + '</td></tr>'
                    html += '<tr><td colspan="2" class="blank"></td><th>Turnout</th>'
                    html += '<td style="text-align:center;">' + '{:,}'.format(res_obj.turnoutN) + '</td>'
                    if isinstance(res_obj.turnoutP,str):
                        html += '<td style="text-align:center;">' + res_obj.turnoutP + '</td></tr>'
                    else:
                        html += '<td style="text-align:center;">' + '{:.1f}'.format(res_obj.turnoutP) + '</td></tr>'
                    html += '</table>'
                else:
                    html += '</table>'
                if res_obj.notes != '':
                    notes += '<div>' + res_obj.notes + '</div>'
                html += notes

                display(widgets.HTML(html))
                
            elif self.country == 'US':
                
                name = properties['name']
                obj = DISTRICT.const_list[name]
                html = '''<style>
                        table {
                          font-family: arial, sans-serif;
                          border-collapse: collapse;
                          width: 60%;
                        }

                        td {
                          border: 0.5px solid #000000;
                          text-align: left;
                          padding: 1px 5px 1px 5px;
                        }

                        th {
                          border: 0.5px solid #000000;
                          background-color: #E7E6E7;
                          text-align: center;
                          padding: 1px 5px 1px 5px; 
                        }

                        .blank {
                          border: none;
                        }
                        </style>'''                        

                html += '<h2>' + name + '</h2>'
                html += '<div><b>State: </b>' + obj.state + '</div>'
                html += '<div><b>Created: </b>' + str(obj.created)
                if obj.abolished != []:
                    for i in range(1,len(obj.abolished)+1):
                        html += '<div><b>Abolished: </b>' + str(obj.abolished[i-1])
                        if len(obj.recreated) >= i:
                            html += '<div><b>Recreated: </b>' + str(obj.recreated[i-1])
                            html += '</div>'

                html += '<h3 style="text-align:left;">' + self.election + ' House Election Results</h3>'

                res_obj = obj.election_list[self.election]
                df = res_obj.results.copy()
                notes = ''
                html += '<table><tr><th colspan="2">Party</th><th>Candidate</th><th>Votes</th><th>Percent</th></tr>'

                for row in df.index:
                    party = US_PARTY.party_list.get(df.Party[row],'')
                    if party == '':
                        party_colour = '#C8C7C9'
                    else:
                        party_colour = party.colour
                    html += '<tr><td style="background-color:' + party_colour + ';">  </td>'
                    html += '<td>' + df.Party[row] + '</td>'
                    html += '<td>' + df.Candidate[row] + '</td>'
                    if df.Votes[row] == 'Unopposed':
                        html += '<td colspan=2 style="text-align:center;">Unopposed</td></tr>'
                    else:
                        html += '<td style="text-align:center;">' + '{:,}'.format(df.Votes[row]) + '</td>'
                        html += '<td style="text-align:center;">' + df.Percent[row] + '</td></tr>'
                        if df.Notes[row] != '':
                            notes += '<div>' + df.Notes[row] + '</div>'
                if not res_obj.unopposed:
                    #majority_percent = res_obj.majority / res_obj.turnoutN * 100
                    #html += '<tr><td colspan="2" class="blank"></td><th>Majority</th>'
                    #html += '<td style="text-align:center;">' + '{:,}'.format(res_obj.majority) + '</td>'
                    #html += '<td style="text-align:center;">' + '{:.1f}'.format(majority_percent) + '</td></tr>'
                    html += '<tr><td colspan="2" class="blank"></td><th>Turnout</th>'
                    html += '<td style="text-align:center;">' + '{:,}'.format(res_obj.turnoutN) + '</td>'
                    #if isinstance(res_obj.turnoutP,str):
                        #html += '<td style="text-align:center;">' + res_obj.turnoutP + '</td></tr>'
                    #else:
                        #html += '<td style="text-align:center;">' + '{:.1f}'.format(res_obj.turnoutP) + '</td></tr>'
                    html += '</table>'
                else:
                    html += '</table>'
                if res_obj.notes != '':
                    notes += '<div>' + res_obj.notes + '</div>'
                html += notes
                
                display(widgets.HTML(html))
            
gui = GUI()

display(gui.gui)



In [16]:
# state_dict={1:'Alabama',2:'Alaska',4:'Arizona',5:'Arkansas',6:'California',8:'Colorado',9:'Connecticut',10:'Delaware',
# 11:'DC',12:'Florida',13:'Georgia',15:'Hawaii',16:'Idaho',17:'Illinois',18:'Indiana',19:'Iowa',20:'Kansas',
# 21:'Kentucky',22:'Louisiana',23:'Maine',24:'Maryland',25:'Massachusetts',26:'Michigan',27:'Minnesota', 
# 28:'Mississippi',29:'Missouri',30:'Montana',31:'Nebraska',32:'Nevada',33:'New Hampshire',34:'New Jersey', 
# 35:'New Mexico',36:'New York',37:'North Carolina',38:'North Dakota',39:'Ohio',40:'Oklahoma',41:'Oregon', 
# 42:'Pennsylvania',44:'Rhode Island',45:'South Carolina',46:'South Dakota',47:'Tennessee',48:'Texas', 
# 49:'Utah',50:'Vermont',51:'Virginia',53:'Washington',54:'West Virginia',55:'Wisconsin',56:'Wyoming',60:'American Samoa',
# 66:'Guam',69:'North Mariana Islands',72:'Puerto Rico',78:'US Virgin Islands'}

# count = 0
# for feature in us_congress_gj.features:
#     state_num = feature['properties']['STATEFP']
#     state_name = state_dict.get(int(state_num),'Unknown')
#     if state_name == 'Unknown':
#         print(state_num)
#     us_congress_gj[count]['properties']['STATE_NAME'] = state_name
#     cd_num = int(feature['properties']['CD116FP'])
#     if cd_num == 0:
#         full_cd = state_name + ' at-large'
#     else:
#         full_cd = state_name + ' ' + str(cd_num)
# #     if state_name == 'DC' or state_name == 'Unknown':
# #         gj[count]['properties']['WINNER'] = 'NONE'
# #     else:
# #         gj[count]['properties']['WINNER'] = full_results.get(full_cd,'')
#     us_congress_gj[count]['properties']['name'] = full_cd
#     count += 1

In [17]:
# const_list = set()
# elect_list = list(ELECTION.elect_list.keys())
# for election in elect_list:
#     const_list = const_list | set([x.name for x in ELECTION.elect_list[election].const_list])
# final_const_list = list(const_list)
# dict1 = {'const':final_const_list}
# for election in elect_list:
#     const_names = [x.name for x in ELECTION.elect_list[election].const_list]
#     checklist = []
#     for const in final_const_list:
#         if const in const_names:
#             checklist.append(1)
#         else:
#             checklist.append(0)
#     dict1.update({election:checklist})
# df = pd.DataFrame(dict1)
# df

In [103]:
# def hover_funcx(event,feature,properties):
#     info_box_overx.value = str(properties) # + " " + str(properties['CD'])

# def get_party_colour(feature):
    
#     state = state_dict[int(feature['properties']['STATEFP'])]
#     try:
#         if '(' in feature['properties']['NAMELSAD']:
#             district = 'at-large'
#         else:
#             district = feature['properties']['NAMELSAD'].split(' ')[-1]
#         name = state + " " + district

#         winner = STATE.state_list[state].district_list[name].results[2020]['Winning Party']

#         if winner == "Democrat":
#             colour = '#1035de'
#         elif winner == "Republican":
#             colour = '#ed220c'
#             #colour = 'YELLOWGREEN'
#         else:
#             colour = '#ffffff'
#     except:
#         print(state)
#         colour = '#ffffff'
#     return {'fillColor':colour,
#             'color':'#000000',
#             'weight':1,
#             'fillOpacity':0.4}

# with open('Data/US/congress20205m.geojson') as f:
#     us_congress_gj = geojson.load(f)

# m = Map(center=(30, -100), zoom=5, layout=Layout(height="1000px",width="1000px"),scroll_wheel_zoom=True)
# # geo_json_can = GeoJSON(
# #     data=can_gj,
# #     style={'opacity': 1, 'fillOpacity': 0.6, 'weight': 1},
# #     hover_style={'color': 'white', 'dashArray': '0', 'fillOpacity': 0.4},
# #     #style_callback=get_party_colour
# # )
# geo_json_us = GeoJSON(
#     data=us_congress_gj,
#     style={'opacity': 1, 'fillOpacity': 0.6, 'weight': 1},
#     hover_style={'color': 'black', 'dashArray': '0', 'fillOpacity': 0.4},
#     style_callback=get_party_colour
# )
# #m.add_layer(geo_json_can)
# m.add_layer(geo_json_us)
# geo_json_us.on_hover(hover_funcx)

# info_box_overx = widgets.Textarea()
# display(info_box_overx)
# display(m)



DC
Puerto Rico
Guam
North Mariana Islands
US Virgin Islands
American Samoa


Textarea(value='')

Map(center=[30, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [ ]:
###################################################### Break Cell ######################################################

In [ ]:
# Old load svg function

#     def load_svgx(self):
        
#         def parse_svg(root,election):
    
#             def transform_path(path,transform,election=''):

#                 if transform[:6] == 'matrix':
#                     a,b,c,d,e,f = [float(i) for i in transform[7:-1].split(',')]
#                     count = 0
#                     for coords in path.vertices:
#                         x = a * coords[0] + c * coords[1] + e
#                         y = b * coords[0] + d * coords[1] + f
#                         path.vertices[count][0] = x
#                         path.vertices[count][1] = y
#                         count += 1

#                 elif transform[:9] == 'translate':
#                     coords_split = [float(i) for i in transform[10:-1].split(',')]
#                     if len(coords_split) == 2:
#                         x_shift, y_shift = coords_split
#                     count = 0
#                     for coords in path.vertices:
#                         x = coords[0] + x_shift
#                         y = coords[1] + y_shift
#                         path.vertices[count][0] = x
#                         path.vertices[count][1] = y
#                         count += 1

#                 elif transform == 'Shetland':
                    
#                     if election in ['1992','1987','1983']:
#                         a,b,c,d,e,f = [0.3,0,0,0.3,60,-190]
#                     elif election in ['2001','1997']:
#                         a,b,c,d,e,f = [0.7,0,0,0.7,-60,-180]
#                     elif election in ['1979','1974 Oct','1974 Feb']:
#                         a,b,c,d,e,f = [0.7,0,0,0.7,0,-280]

#                     count = 0
#                     for coords in path.vertices:
#                         x = a * coords[0] + c * coords[1] + e
#                         y = b * coords[0] + d * coords[1] + f
#                         path.vertices[count][0] = x
#                         path.vertices[count][1] = y
#                         count += 1 

#                 return path

#             def check_element(paths, names, element, transform, election):

#                 if 'transform' in element.keys():
#                     transform = element.attrib['transform']

#                 if element.tag != '{http://www.w3.org/2000/svg}path':
#                     for e in element:
#                         paths, names = check_element(paths,names,e,transform,election)
#                 else:
#                     if 'd' in element.keys() and element.attrib['d'] != '':
                        
#                         path = element.attrib['d']
#                         if path[-1] not in ('z','Z') and path[-2] not in ('z','Z'):
#                             path = path + 'z'
#                         try:
#                             parsed = parse_path(path)
#                         except:
#                             print(element.attrib['id'])
#                             parsed = parse_path(path)
#                         names.append(element.attrib['id'])
#                         if element.attrib['id'] == 'Orkney and Shetland':
#                             parsed = transform_path(parsed,'Shetland',election=election)
#                         elif transform:
#                             parsed = transform_path(parsed,transform)
#                         if parsed:
#                             paths.append(parsed)
                        
#                         return paths, names

#                 return paths, names

#             tree = etree.parse(filename)
#             root = tree.getroot()
#             paths = []
#             names = []

#             for element in root:
#                 transform = None
#                 paths, names = check_element(paths, names, element,transform,election)

#             return paths, names

#         def convert_paths(paths, names, election):
            
#             def convert_path_into_polygons(path,pg_x,pg_y,count=-1):

#                 if count == -1:

#                     vertices = path.vertices
#                     codes = path.codes

#                     cur_pg = []

#                     for i in range(0,len(vertices)):
#                         if codes[i] == 79:
#                             pg_x.append([[c[0] for c in cur_pg]])
#                             pg_y.append([[c[1]*-1 for c in cur_pg]]) # multiply by -1 to invert
#                             cur_pg = []
#                         else:
#                             cur_pg.append(vertices[i])

#                 elif names[count] == 'Milton Keynes North East':

#                     vertices = []
#                     vertices += list(path.vertices[:6])
#                     vertices += list(paths[count+1].vertices[1:10])
#                     vertices += list(path.vertices[15:])
#                     codes = [1,]
#                     codes += [2] * (len(vertices)-2) 
#                     codes.append(79)

#                     cur_pg = []
#                     names[count] = 'Milton Keynes'

#                     for i in range(0,len(vertices)):
#                         if codes[i] == 79:
#                             pg_x.append([[c[0] for c in cur_pg]])
#                             pg_y.append([[c[1]*-1 for c in cur_pg]]) # multiply by -1 to invert
#                             cur_pg = []
#                         else:
#                             cur_pg.append(vertices[i])

#                 return pg_x, pg_y
            
#             xs = []
#             ys = []
#             x_temp = []
#             y_temp = []
#             anomalies = {}
#             count = 0
            
#             for path in paths:
                
#                 if 'Milton' in names[count] and election in ('1987','1983'):
#                     x_temp, y_temp = convert_path_into_polygons(path,x_temp,y_temp,count)
#                     if names[count] == 'Milton Keynes':
#                         xs.append(x_temp)
#                         ys.append(y_temp)
#                     x_temp = []
#                     y_temp = []
#                 elif election in ('1992','1987','1983') and names[count] in ('path4893','Glasgow Central'):
#                     anomalies[names[count]] = path
#                 else:    
#                     x_temp, y_temp = convert_path_into_polygons(path,x_temp,y_temp)
#                     if count == len(paths)-1 or names[count+1] != names[count]:       
#                         xs.append(x_temp)
#                         ys.append(y_temp)
#                         x_temp = []
#                         y_temp = []

#                 count += 1
            
#             return xs, ys, anomalies
        
#         def correct_glasgow(names,xs,ys,anomalies):
    
#             names.remove('Glasgow Central')
#             names.remove('path4893')
#             names.append('Glasgow Central')

#             vertices = []
#             vertices += list(anomalies['path4893'].vertices[0:11])
#             vertices += list(anomalies['Glasgow Central'].vertices[0:6])
#             vertices += list(anomalies['path4893'].vertices[16:])
#             codes = [1,]
#             codes += [2] * (len(vertices)-2) 
#             codes.append(79)

#             cur_pg = []

#             for i in range(0,len(vertices)):
#                 if codes[i] == 79:
#                     xs.append([[[c[0] for c in cur_pg]]])
#                     ys.append([[[c[1]*-1 for c in cur_pg]]]) # multiply by -1 to invert
#                     cur_pg = []
#                 else:
#                     cur_pg.append(vertices[i])

#             return names, xs, ys

#         def check_for_holes(xs, ys, election):
    
#             const_holes = {('1983','1987','1992'):
#                                {'PENRITH AND THE BORDER':'CARLISLE',
#                                 'WANSDYKE':'BATH',
#                                 'CIRENCESTER AND TEWKESBURY':'CHELTENHAM'},
#                        ('1997','2001'):
#                                {'NORTH ESSEX':'COLCHESTER'}
#                       }

#             for key in const_holes.keys():

#                 if election in key:

#                     for const in const_holes[key].keys():

#                         count = 0
#                         outer = 0
#                         inner = 0
#                         for name in names:
#                             if name.upper() == const:
#                                 outer = count
#                             elif name.upper() == const_holes[key][const]:
#                                 inner = count
#                             count += 1

#                         xs[outer][0].append(xs[inner][0][0])
#                         ys[outer][0].append(ys[inner][0][0])

#             return xs, ys
        
#         self.svg_output.clear_output()
        
#         with self.svg_output:
            
#             load = widgets.HTML("<br><br><br>Loading...")
#             display(load)
            
#             if self.country == 'US' and self.us_election == 'Senate':
#                 filename = "SVGs/" + self.ECLASS.elect_list[self.election].state_svg
#             else: 
#                 filename = "SVGs/" + self.ECLASS.elect_list[self.election].svg

#             tree = etree.parse(filename)
#             root = tree.getroot()
#             path_elems = root.findall('.//{http://www.w3.org/2000/svg}path')
#             paths, names = parse_svg(root, self.election)

#             xs, ys, anomalies = convert_paths(paths, names, self.election)
            
#             if self.election in ('1987','1983'):
#                 names.remove('Milton Keynes South West')
#             if self.election in ('1992','1987','1983'):
#                 names, xs, ys = correct_glasgow(names,xs,ys,anomalies)

#             xs, ys = check_for_holes(xs,ys,self.election)

#             for i in range(len(names)-1,-1,-1):
#                 if i != 0 and names[i]==names[i-1]:
#                     names.pop(i)

#             colours = self.get_colours(names, self.election)
            
#             results = []
#             if self.country == 'UK':
#                 for name in names:
#                     if name in CONSTITUENCY.const_list.keys():
#                         results.append(CONSTITUENCY.const_list[name].election_list[self.election].results.to_dict())
#                     else:
#                         results.append(CONSTITUENCY.previous_list[name].election_list[self.election].results.to_dict())
#             else:
#                 for name in names:
#                     if name != 'Washington DC' and name != 'frames':
#                         try:
#                             results.append(STATE.senate_list[name].senate_results[self.election][0].results.to_dict())
#                         except:
#                             print(name)
                        
#             count = 0
#             for const in results:
#                 for key in const.keys():
#                     for subkey in const[key].keys():
#                         results[count][key][subkey] = str(results[count][key][subkey])
#                 count += 1
            
#             data=dict(
#                 x=xs,
#                 y=ys,
#                 name=names,
#                 colours=colours,
#                 results = results
#             )

#             cds = ColumnDataSource(data)

#             TOOLS = "pan,wheel_zoom,reset,hover,save"

#             p = figure(
#                 title="General Election " + self.election, tools=TOOLS,
#                 x_axis_location=None, y_axis_location=None,
#                 tooltips=[("Name", "@name")],
#                 aspect_ratio=0.7)

#             patch_renderer = p.multi_polygons(xs="x",ys="y",line_width=1,color="colours",line_color='black',
#                                               name="names",source=cds)
            
#             p.hover.point_policy = "follow_mouse"

#             indicator_div = Div(text="",min_width=700)
#             layout = bkCol(bkRow(p, indicator_div),width_policy="max")
            
#             tap_tool = TapTool(renderers=[patch_renderer])
#             p.add_tools(tap_tool)
#             display_text = self.get_bokeh_code()
#             patch_indicator_callback = CustomJS(args=dict(cds=cds, div=indicator_div, election=self.election), 
#                                                 code=display_text)
            
#             cds.selected.js_on_change('indices', patch_indicator_callback)
            
#             load.close()
#             show(layout)


In [108]:
# # Hex data

# hex_dict = {
#             'UK':{'df':uk_hex_df,'Name':'Constituency'},
# #              'US':{'elections':{'2020':'HexWDC',},
# #                    'df':us_hex_df,
# #                    'Name':'State'},
# #             'Canada':{'2021':hex_df}
#            }

In [ ]:
# Old SVG

# def parse_svg(filename):
    
#     def transform_path(path,transform):
        
#         if transform[:6] == 'matrix':
#             a,b,c,d,e,f = [float(i) for i in transform[7:-1].split(',')]
#             count = 0
#             for coords in path.vertices:
#                 x = a * coords[0] + c * coords[1] + e
#                 y = b * coords[0] + d * coords[1] + f
#                 path.vertices[count][0] = x
#                 path.vertices[count][1] = y
#                 count += 1
        
#         elif transform[:9] == 'translate':
#             x_shift, y_shift = [float(i) for i in transform[10:-1].split(',')]
#             count = 0
#             for coords in path.vertices:
#                 x = coords[0] + x_shift
#                 y = coords[1] + y_shift
#                 path.vertices[count][0] = x
#                 path.vertices[count][1] = y
#                 count += 1
        
#         elif transform == 'Shetland':
#             a= 0.3
#             b = 0
#             c = 0
#             d = 0.3
#             e = 60
#             f = -190
#             count = 0
#             for coords in path.vertices:
#                 x = a * coords[0] + c * coords[1] + e
#                 y = b * coords[0] + d * coords[1] + f
#                 path.vertices[count][0] = x
#                 path.vertices[count][1] = y
#                 count += 1      
                
#         elif transform == 'Shetland97':
#             a= 0.7
#             b = 0
#             c = 0
#             d = 0.7
#             e = -60
#             f = -180
#             count = 0
#             for coords in path.vertices:
#                 x = a * coords[0] + c * coords[1] + e
#                 y = b * coords[0] + d * coords[1] + f
#                 path.vertices[count][0] = x
#                 path.vertices[count][1] = y
#                 count += 1   
            
#         return path
    
#     def check_element(paths, element, transform):
        
#         if 'transform' in element.keys():
#             transform = element.attrib['transform']
        
#         if element.tag != '{http://www.w3.org/2000/svg}path':
#             try:
#                 for e in element:
#                     paths = check_element(paths,e,transform)
#             except Exception as e:
#                 print(e,element.tag)
#         else:
#             if 'd' in element.keys() and element.attrib['d'] != '':
#                 #print('creating path')
#                 path = element.attrib['d']
#                 parsed = parse_path(path)
#                 if element.attrib['id'] == 'Orkney_and_Shetland':
#                     parsed = transform_path(parsed,'Shetland')
#                 elif element.attrib['id'] == 'Orkney and Shetland':
#                     parsed = transform_path(parsed,'Shetland97')
#                 elif transform:
#                     parsed = transform_path(parsed,transform)
#                 if parsed:
#                     paths.append(parsed)
#                 return paths
            
#         return paths
    
#     tree = etree.parse(filename)
#     root = tree.getroot()
#     width = int(re.match(r'\d+', root.attrib['width']).group())
#     height = int(re.match(r'\d+', root.attrib['height']).group())
#     paths = []
    
#     for element in root:
#         transform = None
#         paths = check_element(paths,element,transform)
       
#     return paths, width, height

# def convert_path_into_polygons(path,pg_x,pg_y,count=-1):
    
#     if count == -1:
        
#         vertices = path.vertices
#         codes = path.codes

#         cur_pg = []

#         for i in range(0,len(vertices)):
#             if codes[i] == 79:
#                 pg_x.append([[c[0] for c in cur_pg]])
#                 pg_y.append([[c[1]*-1 for c in cur_pg]]) # multiply by -1 to invert
#                 cur_pg = []
#             else:
#                 cur_pg.append(vertices[i])
                
#     elif names[count] == 'Milton Keynes North East':
        
#         vertices = []
#         vertices += list(path.vertices[:6])
#         vertices += list(paths[0][count+1].vertices[1:10])
#         vertices += list(path.vertices[15:])
#         codes = [1,]
#         codes += [2] * (len(vertices)-2) 
#         codes.append(79)
        
#         cur_pg = []
#         names[count] = 'Milton Keynes'

#         for i in range(0,len(vertices)):
#             if codes[i] == 79:
#                 pg_x.append([[c[0] for c in cur_pg]])
#                 pg_y.append([[c[1]*-1 for c in cur_pg]]) # multiply by -1 to invert
#                 cur_pg = []
#             else:
#                 cur_pg.append(vertices[i])
    
#     return pg_x, pg_y

# def check_for_holes(xs, ys):
    
#     const_holes = {('1983','1987','1992'):
#                        {'PENRITH AND THE BORDER':'CARLISLE',
#                         'WANSDYKE':'BATH',
#                         'CIRENCESTER AND TEWKESBURY':'CHELTENHAM'},
#                ('1997','2001'):
#                        {'NORTH ESSEX':'COLCHESTER'}
#               }
    
#     for key in const_holes.keys():
        
#         if election in key:
            
#             for const in const_holes[key].keys():
                
#                 count = 0
#                 outer = 0
#                 inner = 0
#                 for name in names:
#                     if name.upper() == const:
#                         outer = count
#                     elif name.upper() == const_holes[key][const]:
#                         inner = count
#                     count += 1

#                 xs[outer][0].append(xs[inner][0][0])
#                 ys[outer][0].append(ys[inner][0][0])
    
#     return xs, ys

# def get_style(elem):
#     style_dict = {}
#     for x in elem.attrib['style'].split(';'):
#         xsplit = x.split(':')
#         style_dict.update({xsplit[0]:xsplit[1]})
#     return style_dict

# def correct_glasgow(names,xs,ys,anomalies):
    
#     names.remove('Glasgow Central')
#     names.remove('path4893')
#     names.append('Glasgow Central')
    
#     vertices = []
#     vertices += list(anomalies['path4893'].vertices[0:11])
#     vertices += list(anomalies['Glasgow Central'].vertices[0:6])
#     vertices += list(anomalies['path4893'].vertices[16:])
#     codes = [1,]
#     codes += [2] * (len(vertices)-2) 
#     codes.append(79)
    
#     cur_pg = []

#     for i in range(0,len(vertices)):
#         if codes[i] == 79:
#             xs.append([[[c[0] for c in cur_pg]]])
#             ys.append([[[c[1]*-1 for c in cur_pg]]]) # multiply by -1 to invert
#             cur_pg = []
#         else:
#             cur_pg.append(vertices[i])
            
#     return names, xs, ys

# election = '1983'
# tree = etree.parse("SVGs/Map1992.svg")
# root = tree.getroot()
# path_elems = root.findall('.//{http://www.w3.org/2000/svg}path')
# names = [elem.attrib['id'] for elem in path_elems]
# paths = parse_svg("SVGs/Map1992.svg")
# #facecolors = [get_style(elem).get('fill', 'none') for elem in path_elems]
# edgecolors = [get_style(elem).get('stroke', 'none') for elem in path_elems]
# linewidths = [get_style(elem).get('stroke_width', 1) for elem in path_elems]

# xs = []
# ys = []
# x_temp = []
# y_temp = []
# count = 0
# colours_to_remove = []
# anomalies = {}

# for path in paths[0]:
#     if 'Milton' in names[count] and election in ('1987','1983'):
#         print(len(xs))
#         x_temp, y_temp = convert_path_into_polygons(path,x_temp,y_temp,count)
#         if names[count] == 'Milton Keynes':
#             xs.append(x_temp)
#             ys.append(y_temp)
#         x_temp = []
#         y_temp = []
#         print(len(xs))
#     elif election in ('1992','1987','1983') and names[count] in ('path4893','Glasgow Central'):
#         anomalies[names[count]] = path
#     else:    
#         x_temp, y_temp = convert_path_into_polygons(path,x_temp,y_temp)
#         if count == len(paths[0])-1 or names[count+1] != names[count]:       
#             xs.append(x_temp)
#             ys.append(y_temp)
#             x_temp = []
#             y_temp = []
#         else:
#             colours_to_remove.append(count)
#     count += 1

# if election in ('1987','1983'):
#     names.remove('Milton Keynes South West')
# if election in ('1992','1987','1983'):
#     names, xs, ys = correct_glasgow(names,xs,ys,anomalies)
    
# xs, ys = check_for_holes(xs,ys)

# # for i in range(len(colours_to_remove)-1,-1,-1):
# #     facecolors.pop(colours_to_remove[i])

# for i in range(len(names)-1,-1,-1):
#     if i != 0 and names[i]==names[i-1]:
#         names.pop(i)
    
# #names = [x.title().replace('_',' ').replace('And','and').replace(' Of ',' of ').replace('Under','under').replace('Weston-Super-Mare','Weston-super-Mare') for x in names]
# facecolors = []
# for name in names:
#     try:
#         if name in CONSTITUENCY.const_list.keys():
#             const_obj = CONSTITUENCY.const_list[name]
#             party = const_obj.election_list[election].party1
#         else:
#             for const in CONSTITUENCY.const_list.values():
#                 if name == const.previous:
#                     const_obj = const
#                     party = const_obj.election_list[election].party1
#         facecolors.append(PARTY.party_colours[party])
#         const_obj = ''
#     except:
#         print(name)
#         0/0

# data=dict(
#     x=xs,
#     y=ys,
#     name=names,
#     colours=facecolors,
#     #results = results
# )

# cds = ColumnDataSource(data)

# TOOLS = "pan,wheel_zoom,reset,hover,save"

# p = figure(
#     title="General Election, " + election, tools=TOOLS,
#     x_axis_location=None, y_axis_location=None,
#     tooltips=[("Name", "@name")],
#     aspect_ratio=0.7)

# p.multi_polygons(xs="x",ys="y",line_width=1,color="colours",line_color='black',name="names",source=cds)

# show(p)

In [250]:
# SVG

# parameters = {'1983':{'shet':574,'scotStart':540,'scotEnd':612},
#               '1987':{'shet':574,'scotStart':540,'scotEnd':612},
#               '1992':{'shet':574,'scotStart':540,'scotEnd':612},}

# def transform_paths(path_obj):
#     a= 0.53385753
#     b = 0
#     c = 0
#     d = 0.5357256
#     e = 7.1570288
#     f = -70.336371
#     new_list = []
#     count = 0
#     for coords in path_obj.vertices:
#         x = a * coords[0] + c * coords[1] + e
#         y = b * coords[0] + d * coords[1] + f
#         path_obj.vertices[count][0] = x
#         path_obj.vertices[count][1] = y
#         count += 1
#     return path_obj

# def transform_shet(path_obj):
#     a= 0.3
#     b = 0
#     c = 0
#     d = 0.3
#     e = 60
#     f = -190
#     new_list = []
#     count = 0
#     for coords in path_obj.vertices:
#         x = a * coords[0] + c * coords[1] + e
#         y = b * coords[0] + d * coords[1] + f
#         path_obj.vertices[count][0] = x
#         path_obj.vertices[count][1] = y
#         count += 1
#     return path_obj

# try:
#     plt.clf()
# except:
#     pass

# #r = requests.get('https://commons.wikimedia.org/wiki/File:UK_General_Election,_1983.svg')
# tree = etree.parse("SVGs/Map1983.svg")
# root = tree.getroot()
# width = int(re.match(r'\d+', root.attrib['width']).group())
# height = int(re.match(r'\d+', root.attrib['height']).group())
# path_elems = root.findall('.//{http://www.w3.org/2000/svg}path')

# paths = []
# count = 0

# names = [elem.attrib['id'] for elem in path_elems]
# scotland = False

# for elem in path_elems:
#     #print(names[count], end='')
#     if elem.attrib['d'] == '':
#         continue
#     if names[count] == 'Roxburgh_and_Berwickshire':
#         scotland = True
#         #pass
#     if names[count] == 'Aberavon':
#         scotland = False
#     #if count >= parameters[election]['scotStart'] and count <= parameters[election]['scotEnd']:
#     if scotland:
#         #if count == parameters[election]['shet']:
#         if names[count] == 'Orkney_and_Shetland':
#             paths.append(transform_shet(parse_path(elem.attrib['d'])))
#         else:
#             paths.append(transform_paths(parse_path(elem.attrib['d'])))
#     else:
#         paths.append(parse_path(elem.attrib['d']))
#     count += 1
    

# facecolors = [get_style(elem).get('fill', 'none') for elem in path_elems]
# edgecolors = [get_style(elem).get('stroke', 'none') for elem in path_elems]
# linewidths = [get_style(elem).get('stroke_width', 1) for elem in path_elems]

# collection = mpl.collections.PathCollection(paths, 
#                                       edgecolors=edgecolors, 
#                                       linewidths=linewidths,
#                                       facecolors=facecolors)

# def update_annot(ind):
#     pos=sc.get_offsets()[ind["ind"][0]]
#     annot.sy=pos
#     text="{},{}".format(" ".join(list(map(str,ind["ind"]))),
#         " ".join([names[n] for n in ind["ind"]]))
#     annot.set_text(text)
#     annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
#     annot.get_bbox_patch().set_alpha(0.4)
    
# def hover(event):
#     vis=annot.get_visible()
#     if event.inaxes == ax:
#         cont, ind = sc.contains(event)
#         if cont:
#             update_annot(ind)
#             annot.set_visible(True)
#             fig.canvas.draw_idle()

# xs = []
# ys = []
# for path in paths:
#     x_p, y_p = convert_path_into_polygons(path)
#     xs.append(x_p)
#     ys.append(y_p)

# # #with plt.ioff():
# # fig = plt.figure(figsize=(12,12))
# # ax = fig.add_subplot(111)
# # collection.set_transform(ax.transData)
# # ax.add_artist(collection)
# # ax.set_xlim([0, width])
# # ax.set_ylim([height, -120])
# # plt.xlim(xmin=-200,xmax=800)
# # ax.axis('off')
# # #ax.imshow(image)
# # annot = ax.annotate("",xy=(0,0),xytext=(20,20),textcoords="offset points",bbox=dict(boxstyle="round",fc="w"),arrowprops=dict(arrowstyle="->"))
# # annot.set_visible(False)            
# # fig.canvas.mpl_connect("motion_notify_event",hover)
# # #disconnect_zoom = zoom_factory(ax)
# # #pan_handler = panhandler(fig)
# # map_output = widgets.Output()
# # map_gui = widgets.VBox([map_output],layout=Layout(width='100%'))
# # display(map_gui)
# # #with map_output:
# # plt.show()


# names = [x.title().replace('_',' ') for x in names]

# data=dict(
#     x=xs,
#     y=ys,
#     name=names,
#     colours=facecolors,
#     #results = results
# )

# cds = ColumnDataSource(data)

# TOOLS = "pan,wheel_zoom,reset,hover,save"

# p = figure(
#     title="General Election, 2019", tools=TOOLS,
#     x_axis_location=None, y_axis_location=None,
#     tooltips=[("Name", "@name")],
#     aspect_ratio=0.5)

# p.multi_polygons(xs="x",ys="y",line_width=1,color="colours",line_color='black',name="names",source=cds)

# # source = ColumnDataSource(dict(xs=xs, ys=ys))

# # plot = Plot(
# #     title=None, width=300, height=300,
# #     min_border=0, toolbar_location=None)

# # glyph = MultiPolygons(xs="xs", ys="ys", line_width=2)
# # plot.add_glyph(source, glyph)

# # xaxis = LinearAxis()
# # plot.add_layout(xaxis, 'below')

# # yaxis = LinearAxis()
# # plot.add_layout(yaxis, 'left')

# # plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
# # plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

# # curdoc().add_root(plot)

# # p.grid.grid_line_color = None
# # p.hover.point_policy = "follow_mouse"

# # patch_renderer  = p.patches('x', 'y', source=cds,
# #           fill_color={"field":"colours"},
# #           fill_alpha=0.7, line_color="white", line_width=0.5)

# show(p)

In [ ]:
# def get_hex(df):
    
#     hex_coords = list(df['2010 Hex'])
#     final_coords = []
#     for hex_string in hex_coords:
#         hex_split = hex_string.split(',')
#         hex_split = [int(x) for x in hex_split]
#         final_coords.append(hex_split)
    
#     # Horizontal cartesian coords
#     hcoord_n = [c[0] for c in final_coords]
#     # Vertical cartersian coords
#     vcoord_n = [2. * np.sin(np.radians(60)) * (c[1] - c[2]) /3. for c in final_coords]
    
#     return hcoord_n, vcoord_n

# def update_annot(ind):
    
#     pos=sc.get_offsets()[ind["ind"][0]]
#     #annot.xy=pos
#     annot.set_x(pos[0])
#     annot.set_y(pos[1])
#     text = names[ind['ind'][0]]
#     info_box_over.value=text
#     annot.set_text(text)
#     annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
#     annot.get_bbox_patch().set_alpha(0.4)
    
# def hex_hover(event):
    
#     global names, sc
#     #outputW.value = str(event) #.xdata) + " " + str(event.ydata)
#     vis=annot.get_visible()
#     if event.inaxes == ax:
#         cont, ind = sc.contains(event)
# #         outputW.value=(str(cont) + " " + str(ind))
#         if cont:
#             #names = list(hex_df['Constituency'])
#             text = names[ind['ind'][0]]
#             #text += str(event)
#             info_box_over.value=text
#             #update_annot(ind)
#             annot.set_visible(True)
#             fig.canvas.draw_idle()


In [89]:
# def switch_election(wid):

#     if map_options.value == 'Map':
#         map_info_box.children = [map_box,info_box]
#         load_map()
#     else:
#         map_info_box.children = [hex_box]
#         display_hex()

# def switch_country(wid):

#     pass

# def display_hex(opw):

#     #hex_output.clear_output()
#     opw.clear_output()

#     #with hex_output:
#     with opw:

#         election = election_option.value
#         names = list(hex_df['Constituency'])
#         colours = get_colours(names,election)
#         bk_xs, bk_ys = get_hex_coords(hex_df)
#         results = [CONSTITUENCY.const_list[x].election_list[election].results.to_dict() for x in names]

#         data=dict(
#             x=bk_xs,
#             y=bk_ys,
#             name=names,
#             colours=colours,
#             results = results
#         )

#         cds = ColumnDataSource(data)

#         TOOLS = "pan,wheel_zoom,reset,hover,save"

#         p = figure(
#             title="General Election, 2019", tools=TOOLS,
#             x_axis_location=None, y_axis_location=None,
#             tooltips=[
#                 ("Name", "@name"), #("Unemployment rate", "@rate%"), ("(Long, Lat)", "($x, $y)")
#             ])

#         p.grid.grid_line_color = None
#         p.hover.point_policy = "follow_mouse"

#         patch_renderer  = p.patches('x', 'y', source=cds,
#                   fill_color={"field":"colours"},
#                   fill_alpha=0.7, line_color="white", line_width=0.5)

#         div = Div(width=400)
#         indicator_div = Div(text="",width=350)
#         layout = bkCol(bkRow(p, indicator_div))

#         tap_tool = TapTool(renderers=[patch_renderer])
#         p.add_tools(tap_tool)

#         patch_indicator_callback = CustomJS(args=dict(cds=cds, div=indicator_div, election=election), code="""
#           div.text = "<h2>" + cds.data['name'][cb_obj.indices] + "</h2><h3>" + election + " General Election Results</h3><br>" + 
#           "<b>Winning Party:</b> " + cds.data['results'][cb_obj.indices]['Party'][0] + "<br><br>" +
#           "<table>" +
#           "<tr>" +
#             "<th>Party</th>" +
#             "<th>Candidate</th>" +
#             "<th>Votes</th>" +
#             "<th>Percent</th>" +
#           "</tr>" +
#           "<tr>"
#           let partyList = cds.data['results'][cb_obj.indices]['Party']
#           for (let i = 0; i < Object.keys(partyList).length; i++) {
#             div.text += "<td>" + cds.data['results'][cb_obj.indices]['Party'][i] + "</td>";
#             div.text += "<td>" + cds.data['results'][cb_obj.indices]['Candidate'][i] + "</td>";
#             div.text += "<td>" + cds.data['results'][cb_obj.indices]['Votes'][i] + "</td>";
#             div.text += "<td>" + cds.data['results'][cb_obj.indices]['Percent'][i] + "</td></tr>";
#             }
#           div.text += "</table>"
#           """)

#         cds.selected.js_on_change('indices', patch_indicator_callback)

#         show(layout)

# title = widgets.HTML("<h2>United Kingdom General Elections</h2>")
# country_option = widgets.ToggleButtons(options=['UK','US','Canada'],
#                                         layout=Layout(margin="10px 0px 20px 0px"),
#                                         button_style='danger')
# election_option = widgets.ToggleButtons(options=['2019','2017','2015','2010'],
#                                 layout=Layout(margin="10px 0px 20px 0px"),
#                                 button_style='warning')
# country_option.observe(switch_country,names='value')
# election_option.observe(switch_election,names='value')
# over_label = widgets.HTML("<b>Constituency at cursor:</b>")
# info_box_over = widgets.Text()
# info_box_click = widgets.Output()
# info_box = widgets.VBox([over_label,info_box_over,info_box_click],layout=Layout(width="40%"))
# map_options = widgets.ToggleButtons(options=['Map','Cartogram'],button_style='info')
# map_options.observe(switch_election,names='value')
# map_output = widgets.VBox()
# hex_output = widgets.Output()
# hex_box = widgets.VBox([map_options,hex_output],layout=Layout(width='100%'))
# m = Map(center=(54, 0), zoom=5, layout=Layout(height="100%",width="100%"),scroll_wheel_zoom=True)
# map_box = widgets.VBox([map_options,m],layout=Layout(width='65%'))
# map_info_box = widgets.HBox([map_box,info_box],layout=Layout(height="1000px",width="100%"))
# gui = widgets.VBox([title,country_option,election_option,map_info_box],layout=Layout(align_items='center'))
# display(gui)

In [14]:
# from bokeh.io import show
# from bokeh.models import LogColorMapper
# from bokeh.palettes import Viridis6 as palette
# from bokeh.plotting import figure
# from bokeh.sampledata.unemployment import data as unemployment
# from bokeh.sampledata.us_counties import data as counties

# palette = tuple(reversed(palette))

# counties = {
#     code: county for code, county in counties.items() if county["state"] == "tx"
# }

# county_xs = [county["lons"] for county in counties.values()]
# county_ys = [county["lats"] for county in counties.values()]

# county_names = [county['name'] for county in counties.values()]
# county_rates = [unemployment[county_id] for county_id in counties]
# color_mapper = LogColorMapper(palette=palette)

# data=dict(
#     x=county_xs,
#     y=county_ys,
#     name=county_names,
#     rate=county_rates,
# )

# TOOLS = "pan,wheel_zoom,reset,hover,save"

# p = figure(
#     title="Texas Unemployment, 2009", tools=TOOLS,
#     x_axis_location=None, y_axis_location=None,
#     tooltips=[
#         ("Name", "@name"), ("Unemployment rate", "@rate%"), ("(Long, Lat)", "($x, $y)")
#     ])
# p.grid.grid_line_color = None
# p.hover.point_policy = "follow_mouse"

# p.patches('x', 'y', source=data,
#           fill_color={'field': 'rate', 'transform': color_mapper},
#           fill_alpha=0.7, line_color="white", line_width=0.5)

# show(p)

In [ ]:
# #Geo file conversion
# from osgeo import gdal

# def shapefile2geojson(infile, outfile, fieldname):
#     '''Translate a shapefile to GEOJSON.'''
#     options = gdal.VectorTranslateOptions(format="GeoJSON", dstSRS="EPSG:4326")
#     gdal.VectorTranslate(outfile, infile, options=options)
#     print("Done")

# gdal.UseExceptions()
# shapefile2geojson('Data/Can/lfed000b21a_e.shp','Data/Can/canED.geojson','')

In [ ]:
# count = 0
# for feature in gj_2005['features']:
#     if feature['properties']['name'] == 'Bristol North West':
#         print(count)
#     count += 1

In [36]:
# # Original data
# import geojson
# with open('Data/GB/gb_const_region.geojson') as f:
#     gj = geojson.load(f)
# with open('Data/NI/2008_Boundaries.geojson') as f:
#     gj_ni = geojson.load(f)

# # Correct GB names
# for feature in gj['features']:
#     feature['properties']['NAME'] = feature['properties']['NAME'].replace(' Boro Const','').replace(' Co Const','').replace(' Burgh Const','').replace('Birmingham,','Birmingham').replace('St.','St')

# # Correct NI names
# for i in gj_ni['features']:
#     i['properties']['PC_NAME'] = i['properties']['PC_NAME'].title()
#     if i['properties']['PC_NAME'] == 'Fermanagh And South Tyrone':
#         i['properties']['PC_NAME'] = 'Fermanagh and South Tyrone'
#     if i['properties']['PC_NAME'] == 'Newry And Armagh':
#         i['properties']['PC_NAME'] = 'Newry and Armagh'

# # Bristol North West cut-off - 80
# North vertex - -2.7022866, 51.5226722
# s_ver = [-2.717705, 51.500679]
# n_ver = [-2.701623, 51.522668]
# bnw_coords = gj_2005['features'][73]['geometry']['coordinates'][0]
# new_coords = []
# stop=True

# for coord in bnw_coords:
#     if coord == s_ver:
#         stop = False
#         new_coords.append(coord)
#     elif coord == n_ver:
#         stop = True
#         new_coords.append(coord)
#     elif not stop:
#         new_coords.append(coord)

# gj_2005['features'][73]['geometry']['coordinates'][0] = new_coords



In [4]:
# # Load election data

# df_2010 = pd.read_excel('UK_Elections.xlsx',sheet_name='2010')
# df_2015 = pd.read_excel('UK_Elections.xlsx',sheet_name='2015')
# df_2017 = pd.read_excel('UK_Elections.xlsx',sheet_name='2017')
# df_2010.fillna('',inplace=True)
# df_2015.fillna('',inplace=True)
# df_2017.fillna('',inplace=True)


In [24]:
# # Add results to geojson

# def add_results(gj,col='Winner',name_field='NAME'):
#     gj['features'][0]['properties']['NAME']
#     for feature in gj['features']:
#         name = feature['properties']['NAME']
#         try:
#             winner10 = df_2010.loc[df_2010[df_2010['Constituency']==name].index[0],col]
#             winner15 = df_2015.loc[df_2015[df_2015['Constituency']==name].index[0],col]
#             winner17 = df_2017.loc[df_2017[df_2017['Constituency']==name].index[0],col]
#             feature['properties']['2010_Winner'] = winner10
#             feature['properties']['2015_Winner'] = winner15
#             feature['properties']['2017_Winner'] = winner17
#         except:
#             print(name)
        
#     return gj

# gj = add_results(gj)
# gj_ni = add_results(gj_ni, name_field='PC_NAME')

In [49]:
# # Write to geojson

# with open('Data/GB/2005_Consts.geojson', 'w') as outfile:
#      geojson.dump(gj_2005, outfile)
    
# with open('Data/GB/ni_with_results.geojson', 'w') as outfile:
#     geojson.dump(gj_ni, outfile)

In [12]:
# count = 0
# names = []
# for feature in gj['features']:
#     feature['properties']['NAME'] = feature['properties']['NAME'].replace(' Boro Const','').replace(' Co Const','').replace(' Burgh Const','').replace('Birmingham,','Birmingham').replace('St.','St')
#     count += 1

In [ ]:
# names = [feature['properties']['NAME'] for feature in gj['features']]
# s = pd.Series(names)
# s.to_excel('UK_Elections_temp.xlsx')

In [6]:

# from xml.dom import minidom
# xmldoc = minidom.parse('UK_General_Election,_1983.svg')
# itemlist = xmldoc.getElementsByTagName('polygon')

# for s in itemlist :
#     for p in s.attributes['points'].value.split():
#         print(p)

# import numpy as np
# import pylab
# for s in itemlist :
#     p = np.vstack(map(lambda x: np.fromstring(x,sep=','), s.attributes['points'].value.split()))
#     p=np.vstack([p,p[0,:]])
#     pylab.plot(p[:,0],p[:,1])
# pylab.show()